# Starbucks Next Best Action (NBA) System

## What This Project Does

Imagine running a coffee shop with thousands of customers. You want to send promotional offers (discounts, BOGO deals), but you face three challenges:

1. **Who should get an offer?** Not everyone needs one
2. **Which offer should each person get?** Different people like different things
3. **Will it make money?** Offers cost money - we need profit, not just sales

**This AI system answers all three questions automatically for each customer.**

---

## The Solution

Instead of sending the same coupon to everyone (wasteful!), this system:
- Identifies which customers actually need an incentive
- Picks the perfect offer type for each person
- Calculates if we'll make profit before sending anything
- Explains WHY each recommendation was made

**Result:** Send the right offer to the right person, or save money by not sending anything when it won't help.

---

## What We Analyzed

- **17,000 customers**
- **138,000+ purchases**
- **76,000+ promotional offers** sent over time
- Customer info: age, income, shopping habits, email engagement

Think of it as: Years of Starbucks receipts, email campaigns, and customer profiles all in one place.

---

## How It Works (5 Steps)

### Step 1: Clean The Data
**What:** Organized messy customer records into usable information  
**Why:** Can't make smart decisions with messy data  
**Like:** Organizing a messy filing cabinet before doing your taxes

### Step 2: Create Smart Metrics
**What:** Built "scores" that predict customer behavior  
**Metrics created:**
- **Recency** - How recently did they shop? (newer = better)
- **Frequency** - How often do they visit? (more = better)  
- **Monetary** - How much do they spend? (higher = better)
- **Engagement** - Do they open/use our offers?

**Like:** Creating a customer report card with grades for "Shopping Frequency" and "Email Engagement"

### Step 3: Group Customers into Segments
**What:** Sorted 17,000 customers into 4 types based on behavior  
**Why:** Different customer types need different strategies  

**The 4 Groups We Found:**
1. **Champions** - Loyal, frequent shoppers (treat them well!)
2. **Promising** - Growing customers (encourage them!)
3. **At Risk** - Used to shop a lot, now fading (win them back!)
4. **Casual** - Occasional visitors (light touch)

**Like:** Organizing friends into groups - gym buddies, movie friends, coffee friends. Each group gets different invitations.

### Step 4: Predict Who Might Leave (Churn)
**What:** AI model predicts which customers will stop visiting  
**Accuracy:** 85% correct  

**Risk Levels:**
- **Low Risk** (56% of customers) - Regular visitors, all good
- **Medium Risk** (32% of customers) - Some warning signs
- **High Risk** (12% of customers) - Likely to leave soon

**Why it matters:** Keeping a customer is easier than winning them back after they leave

**Like:** A health check-up - we spot problems before they get serious

### Step 5: Recommend Best Action for Each Person
**What:** AI picks the best offer (or no offer) for maximum profit  

**Options:**
- **BOGO** (Buy One Get One) - Costs us $5
- **Discount** - Costs us $3
- **Informational** (just news) - Costs us $0.50
- **No Offer** - Sometimes the best move is doing nothing!

**Final Recommendations:**
- 30,499 customers → Send BOGO
- 30,345 customers → Send Discount  
- 15,235 customers → Send Info only
- Rest → Don't send anything (they'll buy anyway or won't respond)

**Like:** A smart assistant saying "Customer A loves freebies, send BOGO. Customer B is price-sensitive, send discount. Customer C will buy anyway, don't waste money on them."

---

## Why This Matters

**Traditional Approach:**
"Send the same 20% discount to everyone!"
- Wastes money on people who'd buy anyway
- Wrong offer for people who want BOGO
- Annoys people who don't want emails

**Our Smart Approach:**
"Send personalized offers only when they'll actually help"
- Right offer for each person
- Only send when profit is positive
- Save money by NOT sending to 25% of customers

**Impact:**
- **Average profit per customer:** $1.67
- **Total expected profit:** $127,000
- **Customers with positive ROI:** 75% (we skip the other 25%)

**Simple math:** Instead of $100,000 in random coupons with $80,000 return, we spend $50,000 in smart coupons with $177,000 return.


---

## What Makes This Special

**Not just a simple coupon system:**

1. **Predicts the future** - Who will leave? Who will buy?
2. **Personalizes** - Different people get different recommendations
3. **Considers profit** - Factors in offer costs, not just revenue
4. **Explains itself** - Shows WHY each recommendation was made (not a black box)
5. **Production ready** - Can be deployed to a real business today

**Most companies just:** "Send 20% off to everyone on their birthday"  
**This system:** "Send BOGO to Sarah (she loves those), nothing to John (he'll buy anyway), and a $5 discount to Maria (she's price-sensitive and at risk of leaving)"

---

## 🎓 Technical Skills Demonstrated

For technical readers, this project shows:
- Data cleaning & engineering (76,000 messy records → clean dataset)
- Feature engineering (RFM analysis, engagement metrics)
- Unsupervised learning (K-Means clustering for segmentation)
- Supervised learning (XGBoost for churn prediction, 85% accuracy)
- Causal inference (Uplift modeling with T-Learner)
- Business analytics (profit optimization, ROI calculation)
- Model explainability (SHAP values)
- Production deployment (saved models, API functions)



In [ ]:
# Install required packages (run this first in Colab)
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm shap imbalanced-learn -q

print(" All packages installed successfully!")

In [ ]:
import os
import shutil
import glob
import textwrap
import pandas as pd

# Setup project directory
PROJECT_DIR = "/content/nba_retention"
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

# Clone the repository with the dataset
!rm -rf Starbucks-Capstone
!git clone --depth 1 https://github.com/reachanihere/Starbucks-Capstone.git

# Confirm the files exist
print("Dataset files:")
!ls -lah Starbucks-Capstone/data

print(f"\n Working directory: {os.getcwd()}")

In [ ]:
"""
Starbucks Customer Segmentation & Next Best Action (NBA) Project
=================================================================
"""

# Core libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning (we'll use these later)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("=" * 80)
print("STARBUCKS CUSTOMER SEGMENTATION & NEXT BEST ACTION PROJECT")
print("=" * 80)
print("\n All libraries imported successfully!")
print(f" Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f" Pandas version: {pd.__version__}")
print(f" NumPy version: {np.__version__}")

In [ ]:
"""
Load the three Starbucks JSON files from the cloned repository
"""

print("=" * 80)
print("LOADING STARBUCKS DATA FILES")
print("=" * 80)

# Define file paths
data_dir = 'Starbucks-Capstone/data'

# Load Portfolio (Offer information)
print("\nLoading portfolio.json...")
portfolio = pd.read_json(f'{data_dir}/portfolio.json', orient='records', lines=True)
print(f"Portfolio loaded: {portfolio.shape[0]} offers × {portfolio.shape[1]} columns")

# Load Profile (Customer demographics)
print("\nLoading profile.json...")
profile = pd.read_json(f'{data_dir}/profile.json', orient='records', lines=True)
print(f"Profile loaded: {profile.shape[0]:,} customers × {profile.shape[1]} columns")

# Load Transcript (Event log)
print("\n Loading transcript.json...")
transcript = pd.read_json(f'{data_dir}/transcript.json', orient='records', lines=True)
print(f" Transcript loaded: {transcript.shape[0]:,} events × {transcript.shape[1]} columns")

print("\n" + "=" * 80)
print(" ALL DATA FILES LOADED SUCCESSFULLY")
print("=" * 80)

In [ ]:
"""
Quick exploration of each dataset
"""

print("=" * 80)
print(" INITIAL DATA EXPLORATION")
print("=" * 80)

print("\n" + "="*80)
print(" PORTFOLIO (Offer Information)")
print("="*80)
display(portfolio.head())
print(f"\nShape: {portfolio.shape}")
print(f"Columns: {portfolio.columns.tolist()}")
print(f"\nData types:\n{portfolio.dtypes}")

print("\n" + "="*80)
print(" PROFILE (Customer Demographics)")
print("="*80)
display(profile.head())
print(f"\nShape: {profile.shape}")
print(f"Columns: {profile.columns.tolist()}")
print(f"\nData types:\n{profile.dtypes}")

print("\n" + "="*80)
print("  TRANSCRIPT (Event Log)")
print("="*80)
display(transcript.head(10))
print(f"\nShape: {transcript.shape}")
print(f"Columns: {transcript.columns.tolist()}")
print(f"\nData types:\n{transcript.dtypes}")

In [ ]:
"""
Check for missing values and data quality issues
"""

print("=" * 80)
print(" DATA QUALITY ASSESSMENT")
print("=" * 80)

print("\n PORTFOLIO - Missing Values:")
print("-" * 40)
missing_portfolio = portfolio.isnull().sum()
print(missing_portfolio[missing_portfolio > 0] if missing_portfolio.sum() > 0 else "No missing values ✓")

print("\n PROFILE - Missing Values:")
print("-" * 40)
missing_profile = profile.isnull().sum()
print(missing_profile)
print(f"\nMissing percentage:")
print((missing_profile / len(profile) * 100).round(2))

print("\n TRANSCRIPT - Missing Values:")
print("-" * 40)
missing_transcript = transcript.isnull().sum()
print(missing_transcript[missing_transcript > 0] if missing_transcript.sum() > 0 else "No missing values ✓")

print("\n" + "=" * 80)
print(" KEY INSIGHTS:")
print("=" * 80)
print(f"• Portfolio: {len(portfolio)} unique offers")
print(f"• Profile: {len(profile):,} customers with {profile['gender'].isnull().sum():,} missing gender")
print(f"• Transcript: {len(transcript):,} total events")
print(f"• Event types: {transcript['event'].nunique()} unique event types")

In [ ]:
"""
Understand the event types in the transcript
"""

print("=" * 80)
print(" EVENT TYPE ANALYSIS")
print("=" * 80)

print("\n Event Type Distribution:")
print("-" * 40)
event_counts = transcript['event'].value_counts()
print(event_counts)

print("\n Event Type Percentages:")
print("-" * 40)
event_pct = (transcript['event'].value_counts(normalize=True) * 100).round(2)
print(event_pct)

# Visualize event distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
event_counts.plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('Event Type Distribution (Count)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Event Type', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, v in enumerate(event_counts):
    ax1.text(i, v + 1000, f'{v:,}', ha='center', fontsize=10)

# Percentage plot
event_pct.plot(kind='bar', ax=ax2, color='coral', edgecolor='black')
ax2.set_title('Event Type Distribution (%)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Event Type', fontsize=12)
ax2.set_ylabel('Percentage (%)', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

# Add percentage labels on bars
for i, v in enumerate(event_pct):
    ax2.text(i, v + 0.5, f'{v:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n Event analysis complete!")

Clean Portfolio Data

In [ ]:
"""
STEP 1.1: Clean and Prepare Portfolio (Offer) Data
"""

print("=" * 80)
print("🧹 CLEANING PORTFOLIO DATA")
print("=" * 80)

portfolio_clean = portfolio.copy()

# Rename 'id' to 'offer_id' for clarity
portfolio_clean = portfolio_clean.rename(columns={
    'id': 'offer_id',
    'difficulty': 'min_spend_required',
    'duration': 'duration_days'
})

# Expand the channels column (it's a list) into separate binary columns
print("\n Processing channels...")
for channel in ['email', 'mobile', 'social', 'web']:
    portfolio_clean[f'channel_{channel}'] = portfolio_clean['channels'].apply(
        lambda x: 1 if channel in x else 0
    )

# Create simplified offer type labels
portfolio_clean['offer_type_simple'] = portfolio_clean['offer_type'].map({
    'bogo': 'BOGO',
    'discount': 'Discount',
    'informational': 'Informational'
})

print("\n Portfolio Data Cleaned!")
print("-" * 80)
display(portfolio_clean[['offer_id', 'offer_type_simple', 'reward',
                         'min_spend_required', 'duration_days',
                         'channel_email', 'channel_mobile', 'channel_social', 'channel_web']].head())

print(f"\n Offer Type Distribution:")
print(portfolio_clean['offer_type_simple'].value_counts())

# Visualize offer types
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Offer type distribution
portfolio_clean['offer_type_simple'].value_counts().plot(
    kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[0].set_title('Offer Type Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Offer Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Reward distribution by offer type
portfolio_clean.groupby('offer_type_simple')['reward'].mean().plot(
    kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[1].set_title('Average Reward by Offer Type', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Offer Type')
axes[1].set_ylabel('Average Reward ($)')
axes[1].tick_params(axis='x', rotation=0)
axes[1].grid(axis='y', alpha=0.3)

# Duration distribution
portfolio_clean.groupby('offer_type_simple')['duration_days'].mean().plot(
    kind='bar', ax=axes[2], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[2].set_title('Average Duration by Offer Type', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Offer Type')
axes[2].set_ylabel('Duration (Days)')
axes[2].tick_params(axis='x', rotation=0)
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

Clean Profile Data

In [ ]:
"""
STEP 1.2: Clean and Prepare Profile (Customer) Data
"""

print("=" * 80)
print("🧹 CLEANING PROFILE DATA")
print("=" * 80)

profile_clean = profile.copy()

# Rename 'id' to 'customer_id' for clarity
profile_clean = profile_clean.rename(columns={'id': 'customer_id'})

# Convert became_member_on to datetime
print("\n Converting membership dates...")
profile_clean['became_member_on'] = pd.to_datetime(
    profile_clean['became_member_on'],
    format='%Y%m%d'
)

# Handle outlier ages (ages like 118 are likely data errors)
print("\n Handling age outliers...")
print(f"Age range before cleaning: {profile_clean['age'].min()} - {profile_clean['age'].max()}")
profile_clean['age_clean'] = profile_clean['age'].apply(
    lambda x: x if (x >= 18 and x <= 100) else np.nan
)
print(f"Age range after cleaning: {profile_clean['age_clean'].min()} - {profile_clean['age_clean'].max()}")

# Create age buckets
profile_clean['age_bucket'] = pd.cut(
    profile_clean['age_clean'],
    bins=[0, 25, 35, 45, 55, 65, 100],
    labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
)

# Handle missing income (impute with median by gender)
print("\n💰 Handling income data...")
profile_clean['income_clean'] = profile_clean.groupby('gender')['income'].transform(
    lambda x: x.fillna(x.median())
)

# Create income buckets
profile_clean['income_bucket'] = pd.cut(
    profile_clean['income_clean'],
    bins=[0, 40000, 60000, 80000, 100000, 150000],
    labels=['<40K', '40-60K', '60-80K', '80-100K', '100K+']
)

# Handle missing gender
profile_clean['gender'] = profile_clean['gender'].fillna('Unknown')

# Calculate member tenure (we'll use a reference date - assume data collection end of July 2018)
REFERENCE_DATE = pd.to_datetime('2018-07-31')
profile_clean['member_tenure_days'] = (
    REFERENCE_DATE - profile_clean['became_member_on']
).dt.days

# Create tenure buckets
profile_clean['tenure_bucket'] = pd.cut(
    profile_clean['member_tenure_days'],
    bins=[0, 365, 730, 1095, 10000],
    labels=['<1yr', '1-2yr', '2-3yr', '3yr+']
)

print("\n Profile Data Cleaned!")
print("-" * 80)
print(f"\nMissing values after cleaning:")
print(profile_clean[['customer_id', 'gender', 'age_clean', 'income_clean',
                     'became_member_on', 'member_tenure_days']].isnull().sum())

print(f"\n Customer Summary:")
print(f"Total customers: {len(profile_clean):,}")
print(f"\nGender distribution:")
print(profile_clean['gender'].value_counts())
print(f"\nAge distribution:")
print(profile_clean['age_bucket'].value_counts().sort_index())
print(f"\nIncome distribution:")
print(profile_clean['income_bucket'].value_counts().sort_index())

Visualize Customer Demographics

In [ ]:
"""
Visualize cleaned customer demographics
"""

print("=" * 80)
print(" CUSTOMER DEMOGRAPHICS VISUALIZATION")
print("=" * 80)

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Gender distribution
profile_clean['gender'].value_counts().plot(
    kind='bar', ax=axes[0, 0], color=['#3498db', '#e74c3c', '#95a5a6'], edgecolor='black'
)
axes[0, 0].set_title('Gender Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Gender')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].grid(axis='y', alpha=0.3)

# Age distribution
profile_clean['age_bucket'].value_counts().sort_index().plot(
    kind='bar', ax=axes[0, 1], color='#9b59b6', edgecolor='black'
)
axes[0, 1].set_title('Age Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Age Group')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# Income distribution
profile_clean['income_bucket'].value_counts().sort_index().plot(
    kind='bar', ax=axes[0, 2], color='#1abc9c', edgecolor='black'
)
axes[0, 2].set_title('Income Distribution', fontsize=12, fontweight='bold')
axes[0, 2].set_xlabel('Income Range')
axes[0, 2].set_ylabel('Count')
axes[0, 2].tick_params(axis='x', rotation=45)
axes[0, 2].grid(axis='y', alpha=0.3)

# Member tenure distribution
profile_clean['tenure_bucket'].value_counts().sort_index().plot(
    kind='bar', ax=axes[1, 0], color='#e67e22', edgecolor='black'
)
axes[1, 0].set_title('Member Tenure Distribution', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Tenure')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)

# Age histogram (continuous)
profile_clean['age_clean'].dropna().hist(ax=axes[1, 1], bins=30, color='#34495e', edgecolor='black')
axes[1, 1].set_title('Age Distribution (Continuous)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Age')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(axis='y', alpha=0.3)

# Income histogram (continuous)
profile_clean['income_clean'].dropna().hist(ax=axes[1, 2], bins=30, color='#16a085', edgecolor='black')
axes[1, 2].set_title('Income Distribution (Continuous)', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Income ($)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Demographics visualization complete!")

Clean Transcript Data

In [ ]:
"""
STEP 1.3: Clean and Prepare Transcript (Event Log) Data
"""

print("=" * 80)
print(" CLEANING TRANSCRIPT DATA")
print("=" * 80)

transcript_clean = transcript.copy()

# Rename 'person' to 'customer_id' for consistency
transcript_clean = transcript_clean.rename(columns={'person': 'customer_id'})

# Expand the 'value' column (it's a dictionary)
print("\n Expanding 'value' column...")
value_expanded = transcript_clean['value'].apply(pd.Series)
transcript_clean = pd.concat([transcript_clean.drop('value', axis=1), value_expanded], axis=1)

print(f"New columns from 'value': {value_expanded.columns.tolist()}")

# Convert time (hours) to actual timestamp
print("\n Converting time to timestamps...")
# Time is in hours since start of experiment (assume start = 2018-01-01)
START_DATE = pd.to_datetime('2018-01-01')
transcript_clean['timestamp'] = START_DATE + pd.to_timedelta(transcript_clean['time'], unit='h')
transcript_clean['days_since_start'] = transcript_clean['time'] / 24

print("\n Transcript Data Cleaned!")
print("-" * 80)
display(transcript_clean.head(10))

print(f"\n Event Summary:")
print(f"Total events: {len(transcript_clean):,}")
print(f"Unique customers: {transcript_clean['customer_id'].nunique():,}")
print(f"Date range: {transcript_clean['timestamp'].min()} to {transcript_clean['timestamp'].max()}")
print(f"Duration: {(transcript_clean['timestamp'].max() - transcript_clean['timestamp'].min()).days} days")

print(f"\n Event Type Breakdown:")
print(transcript_clean['event'].value_counts())

Analyze Event Patterns

In [ ]:
"""
Analyze temporal patterns in the event log
"""

print("=" * 80)
print("TEMPORAL EVENT ANALYSIS")
print("=" * 80)

# Events over time
events_by_day = transcript_clean.groupby(
    transcript_clean['timestamp'].dt.date
).size().reset_index(name='event_count')
events_by_day['timestamp'] = pd.to_datetime(events_by_day['timestamp'])

# Plot events over time
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Total events over time
axes[0].plot(events_by_day['timestamp'], events_by_day['event_count'],
             color='#3498db', linewidth=2)
axes[0].set_title('Total Events Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Events')
axes[0].grid(True, alpha=0.3)
axes[0].fill_between(events_by_day['timestamp'], events_by_day['event_count'],
                      alpha=0.3, color='#3498db')

# Events by type over time
events_by_type_time = transcript_clean.groupby(
    [transcript_clean['timestamp'].dt.date, 'event']
).size().reset_index(name='count')
events_by_type_time['timestamp'] = pd.to_datetime(events_by_type_time['timestamp'])

for event_type in transcript_clean['event'].unique():
    data = events_by_type_time[events_by_type_time['event'] == event_type]
    axes[1].plot(data['timestamp'], data['count'], label=event_type, linewidth=2, marker='o', markersize=3)

axes[1].set_title('Events by Type Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Events')
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Temporal analysis complete!")

Define Decision Framework

In [ ]:
"""
STEP 1.4: Define Decision Framework and Business Rules
"""

print("=" * 80)
print(" DECISION FRAMEWORK DEFINITION")
print("=" * 80)

print("""
DECISION FRAMEWORK FOR NEXT BEST ACTION
========================================

 DECISION MOMENT:
   When an offer is received by a customer, that timestamp becomes "today"
   We use only data BEFORE this moment to make predictions

 ACTIONS AVAILABLE:
   1. BOGO (Buy One Get One)
   2. Discount (Percentage off)
   3. Informational (Awareness only, no reward)
   4. No Offer (Control group)

 OUTCOMES TO MEASURE:

   PRIMARY OUTCOMES (within evaluation windows):

   1. CONVERSION (Short-term - 14 days):
      - Did customer complete the offer OR make any purchase?
      - Binary: Yes (1) / No (0)

   2. RETENTION (Long-term - 30 days):
      - Number of transactions in next 30 days
      - Binary: Active (1) / Inactive (0)

 BUSINESS GOAL:
   Maximize incremental profit = Revenue uplift - Offer cost
""")

# Define evaluation windows
CONVERSION_WINDOW_DAYS = 14
RETENTION_WINDOW_DAYS = 30

print(f"\n  EVALUATION WINDOWS:")
print(f"   • Conversion window: {CONVERSION_WINDOW_DAYS} days")
print(f"   • Retention window: {RETENTION_WINDOW_DAYS} days")

# Define offer costs (assumptions for profit calculation)
OFFER_COSTS = {
    'BOGO': 5.0,           # Cost of giving away free item
    'Discount': 3.0,       # Average discount amount
    'Informational': 0.5,  # Minimal cost for sending notification
    'No Offer': 0.0        # No cost
}

print(f"\n ASSUMED OFFER COSTS:")
for offer_type, cost in OFFER_COSTS.items():
    print(f"   • {offer_type:20s}: ${cost:.2f}")

# Store these as constants for later use
print("\n Decision framework defined!")
print("\n These parameters will be used throughout the modeling process")

Create Offer-Customer Interaction Mapping

In [ ]:
"""
STEP 1.5: Create Offer-Customer Interaction Mapping
This creates our "decision moments" - every time an offer was received
"""

print("=" * 80)
print(" CREATING OFFER-CUSTOMER INTERACTION MAPPING")
print("=" * 80)

# Filter for 'offer received' events - these are our decision moments
offers_received = transcript_clean[
    transcript_clean['event'] == 'offer received'
].copy()

print(f"\n Total 'offer received' events: {len(offers_received):,}")
print(f"   Unique customers who received offers: {offers_received['customer_id'].nunique():,}")
print(f"   Date range: {offers_received['timestamp'].min()} to {offers_received['timestamp'].max()}")

# Check what columns we actually have after expanding 'value'
print(f"\n Available columns in offers_received:")
print(offers_received.columns.tolist())

# The column might be 'offer id' (with space) - let's check and handle it
if 'offer id' in offers_received.columns:
    # Just rename it, don't drop the existing offer_id if it exists
    offers_received['offer_id_from_value'] = offers_received['offer id']
    print(" Created 'offer_id_from_value' from 'offer id' column")

    # Use this column for merging
    offer_id_col = 'offer_id_from_value'
elif 'offer_id' in offers_received.columns:
    offer_id_col = 'offer_id'
    print(" Using existing 'offer_id' column")
else:
    print(" ERROR: Cannot find offer id column!")
    print(f"Available columns: {offers_received.columns.tolist()}")

# Drop the old offer_id column if it exists and is causing issues
if 'offer_id' in offers_received.columns and offer_id_col == 'offer_id_from_value':
    offers_received = offers_received.drop(columns=['offer_id'])
    print(" Dropped duplicate 'offer_id' column")

# Rename our working column to offer_id
if offer_id_col == 'offer_id_from_value':
    offers_received = offers_received.rename(columns={'offer_id_from_value': 'offer_id'})

# Now merge with portfolio to get offer details
offers_received = offers_received.merge(
    portfolio_clean[['offer_id', 'offer_type_simple', 'reward', 'min_spend_required', 'duration_days']],
    on='offer_id',
    how='left',
    suffixes=('', '_portfolio')  # Add suffix to avoid column name conflicts
)

print(f"\n Merged with portfolio data")
print(f"\n Offers received by type:")
print(offers_received['offer_type_simple'].value_counts())

# Create a unique identifier for each offer instance
# (same customer can receive same offer multiple times)
offers_received['offer_instance_id'] = (
    offers_received['customer_id'].astype(str) + '_' +
    offers_received['offer_id'].astype(str) + '_' +
    offers_received['timestamp'].astype(str)
)

print(f"\n Created {len(offers_received):,} unique offer instances")

# Preview the offer instances
print("\n Sample of offer instances:")
display(offers_received[['customer_id', 'timestamp', 'offer_id', 'offer_type_simple',
                         'reward', 'min_spend_required', 'duration_days']].head(10))

print(f"\n Final columns in offers_received:")
print(offers_received.columns.tolist())

Analyze Offer Received Patterns

In [ ]:
"""
Analyze patterns in offer distribution
"""

print("=" * 80)
print(" OFFER DISTRIBUTION ANALYSIS")
print("=" * 80)

# How many offers does each customer receive on average?
offers_per_customer = offers_received.groupby('customer_id').size()

print(f"\n Offers per customer statistics:")
print(f"   Mean: {offers_per_customer.mean():.2f}")
print(f"   Median: {offers_per_customer.median():.0f}")
print(f"   Min: {offers_per_customer.min()}")
print(f"   Max: {offers_per_customer.max()}")
print(f"   Std: {offers_per_customer.std():.2f}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Distribution of offers per customer
axes[0].hist(offers_per_customer, bins=30, color='#3498db', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Offers per Customer', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Number of Offers Received')
axes[0].set_ylabel('Number of Customers')
axes[0].axvline(offers_per_customer.mean(), color='red', linestyle='--',
                linewidth=2, label=f'Mean: {offers_per_customer.mean():.1f}')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Offers by type over time
offers_by_type_date = offers_received.groupby(
    [offers_received['timestamp'].dt.date, 'offer_type_simple']
).size().reset_index(name='count')
offers_by_type_date['date'] = pd.to_datetime(offers_by_type_date['timestamp'])

for offer_type in offers_received['offer_type_simple'].dropna().unique():
    data = offers_by_type_date[offers_by_type_date['offer_type_simple'] == offer_type]
    axes[1].plot(data['date'], data['count'], label=offer_type, linewidth=2, marker='o', markersize=4)

axes[1].set_title('Offers Sent by Type Over Time', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Offers Sent')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# Offer type distribution (pie chart)
offer_type_counts = offers_received['offer_type_simple'].value_counts()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
axes[2].pie(offer_type_counts.values, labels=offer_type_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, wedgeprops={'edgecolor': 'black', 'linewidth': 1.5})
axes[2].set_title('Offer Type Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Offer distribution analysis complete!")

Create Offer Viewed and Completed Events

In [ ]:
"""
Extract offer viewed and offer completed events
These will be used to calculate conversion outcomes
"""

print("=" * 80)
print(" EXTRACTING OFFER VIEWED AND COMPLETED EVENTS")
print("=" * 80)

# Offer viewed events
offers_viewed = transcript_clean[
    transcript_clean['event'] == 'offer viewed'
].copy()

# Rename column if needed
if 'offer id' in offers_viewed.columns:
    offers_viewed = offers_viewed.rename(columns={'offer id': 'offer_id'})

print(f"\n  Offer viewed events: {len(offers_viewed):,}")
print(f"   Unique customers: {offers_viewed['customer_id'].nunique():,}")

# Offer completed events
offers_completed = transcript_clean[
    transcript_clean['event'] == 'offer completed'
].copy()

# Rename column if needed
if 'offer id' in offers_completed.columns:
    offers_completed = offers_completed.rename(columns={'offer id': 'offer_id'})

print(f"\n Offer completed events: {len(offers_completed):,}")
print(f"   Unique customers: {offers_completed['customer_id'].nunique():,}")

# View rate: % of received offers that were viewed
total_received = len(offers_received)
total_viewed = len(offers_viewed)
view_rate = (total_viewed / total_received) * 100

print(f"\n Overall view rate: {view_rate:.2f}%")

# Completion rate: % of received offers that were completed
total_completed = len(offers_completed)
completion_rate = (total_completed / total_received) * 100

print(f" Overall completion rate: {completion_rate:.2f}%")

# Completion rate given view
if total_viewed > 0:
    completion_given_view = (total_completed / total_viewed) * 100
    print(f" Completion rate (given view): {completion_given_view:.2f}%")

# Visualize the funnel
funnel_data = pd.DataFrame({
    'Stage': ['Received', 'Viewed', 'Completed'],
    'Count': [total_received, total_viewed, total_completed]
})

fig, ax = plt.subplots(figsize=(10, 6))
colors_funnel = ['#3498db', '#2ecc71', '#e74c3c']
bars = ax.barh(funnel_data['Stage'], funnel_data['Count'], color=colors_funnel, edgecolor='black')

# Add percentage labels
for i, (stage, count) in enumerate(zip(funnel_data['Stage'], funnel_data['Count'])):
    percentage = (count / total_received) * 100
    ax.text(count + 1000, i, f'{count:,} ({percentage:.1f}%)',
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Number of Events', fontsize=12)
ax.set_title('Offer Engagement Funnel', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
"""
Extract transaction (purchase) events
These will be used for calculating retention and spending outcomes
"""

print("=" * 80)
print(" EXTRACTING TRANSACTION EVENTS")
print("=" * 80)

# Transaction events
transactions = transcript_clean[
    transcript_clean['event'] == 'transaction'
].copy()

print(f"\n Total transactions: {len(transactions):,}")
print(f"   Unique customers who transacted: {transactions['customer_id'].nunique():,}")
print(f"   Date range: {transactions['timestamp'].min()} to {transactions['timestamp'].max()}")

# Transaction statistics
print(f"\n Transaction Amount Statistics:")
print(f"   Mean: ${transactions['amount'].mean():.2f}")
print(f"   Median: ${transactions['amount'].median():.2f}")
print(f"   Min: ${transactions['amount'].min():.2f}")
print(f"   Max: ${transactions['amount'].max():.2f}")
print(f"   Total revenue: ${transactions['amount'].sum():,.2f}")

# Transactions per customer
txns_per_customer = transactions.groupby('customer_id').size()
print(f"\n Transactions per customer:")
print(f"   Mean: {txns_per_customer.mean():.2f}")
print(f"   Median: {txns_per_customer.median():.0f}")
print(f"   Min: {txns_per_customer.min()}")
print(f"   Max: {txns_per_customer.max()}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Transaction amount distribution
axes[0].hist(transactions['amount'], bins=50, color='#2ecc71', edgecolor='black', alpha=0.7)
axes[0].set_title('Transaction Amount Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Transaction Amount ($)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(transactions['amount'].mean(), color='red', linestyle='--',
                linewidth=2, label=f'Mean: ${transactions["amount"].mean():.2f}')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Transactions per customer distribution
axes[1].hist(txns_per_customer, bins=30, color='#9b59b6', edgecolor='black', alpha=0.7)
axes[1].set_title('Transactions per Customer Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Transactions')
axes[1].set_ylabel('Number of Customers')
axes[1].axvline(txns_per_customer.mean(), color='red', linestyle='--',
                linewidth=2, label=f'Mean: {txns_per_customer.mean():.1f}')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# Transactions over time
txns_by_date = transactions.groupby(transactions['timestamp'].dt.date).agg({
    'amount': ['count', 'sum']
}).reset_index()
txns_by_date.columns = ['date', 'count', 'revenue']
txns_by_date['date'] = pd.to_datetime(txns_by_date['date'])

ax2 = axes[2]
ax2.plot(txns_by_date['date'], txns_by_date['count'],
         color='#e74c3c', linewidth=2, label='Transaction Count')
ax2.set_xlabel('Date')
ax2.set_ylabel('Transaction Count', color='#e74c3c')
ax2.tick_params(axis='y', labelcolor='#e74c3c')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

ax2_twin = ax2.twinx()
ax2_twin.plot(txns_by_date['date'], txns_by_date['revenue'],
              color='#3498db', linewidth=2, linestyle='--', label='Revenue')
ax2_twin.set_ylabel('Revenue ($)', color='#3498db')
ax2_twin.tick_params(axis='y', labelcolor='#3498db')

axes[2].set_title('Transactions and Revenue Over Time', fontsize=12, fontweight='bold')

# Combine legends
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax2_twin.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

print("\n Transaction analysis complete!")

Customer Activity Summary

In [ ]:
"""
Create a comprehensive customer activity summary
"""

print("=" * 80)
print(" CUSTOMER ACTIVITY SUMMARY")
print("=" * 80)

# Aggregate all customer activities
customer_summary = pd.DataFrame({
    'customer_id': profile_clean['customer_id']
})

# Merge with profile data
customer_summary = customer_summary.merge(
    profile_clean[['customer_id', 'gender', 'age_clean', 'income_clean',
                   'became_member_on', 'member_tenure_days', 'age_bucket',
                   'income_bucket', 'tenure_bucket']],
    on='customer_id',
    how='left'
)

# Add offer statistics
offer_stats = offers_received.groupby('customer_id').agg({
    'offer_instance_id': 'count',
    'offer_type_simple': lambda x: x.mode()[0] if len(x.mode()) > 0 else None
}).rename(columns={
    'offer_instance_id': 'total_offers_received',
    'offer_type_simple': 'most_common_offer_type'
}).reset_index()
customer_summary = customer_summary.merge(offer_stats, on='customer_id', how='left')

# Add view statistics
view_stats = offers_viewed.groupby('customer_id').size().reset_index(name='total_offers_viewed')
customer_summary = customer_summary.merge(view_stats, on='customer_id', how='left')

# Add completion statistics
completion_stats = offers_completed.groupby('customer_id').size().reset_index(name='total_offers_completed')
customer_summary = customer_summary.merge(completion_stats, on='customer_id', how='left')

# Add transaction statistics
txn_stats = transactions.groupby('customer_id').agg({
    'amount': ['count', 'sum', 'mean', 'max']
}).reset_index()
txn_stats.columns = ['customer_id', 'total_transactions', 'total_spent', 'avg_transaction', 'max_transaction']
customer_summary = customer_summary.merge(txn_stats, on='customer_id', how='left')

# Fill NaN values with 0 for counts
count_columns = ['total_offers_received', 'total_offers_viewed', 'total_offers_completed',
                 'total_transactions', 'total_spent', 'avg_transaction', 'max_transaction']
customer_summary[count_columns] = customer_summary[count_columns].fillna(0)

# Calculate engagement rates
customer_summary['offer_view_rate'] = np.where(
    customer_summary['total_offers_received'] > 0,
    customer_summary['total_offers_viewed'] / customer_summary['total_offers_received'],
    0
)

customer_summary['offer_completion_rate'] = np.where(
    customer_summary['total_offers_received'] > 0,
    customer_summary['total_offers_completed'] / customer_summary['total_offers_received'],
    0
)

print(f"\n Customer summary created for {len(customer_summary):,} customers")
print(f"\nSummary statistics:")
print(customer_summary[['total_offers_received', 'total_offers_viewed',
                        'total_offers_completed', 'total_transactions',
                        'total_spent']].describe())

# Preview
print("\n Sample customer summaries:")
display(customer_summary.head(10))

Save Cleaned Data

In [ ]:
"""
Save all cleaned datasets for next steps
"""

print("=" * 80)
print(" SAVING CLEANED DATASETS")
print("=" * 80)

# Create a data folder if it doesn't exist
os.makedirs('cleaned_data', exist_ok=True)

# Save all cleaned datasets
portfolio_clean.to_csv('cleaned_data/portfolio_clean.csv', index=False)
print(" Saved: portfolio_clean.csv")

profile_clean.to_csv('cleaned_data/profile_clean.csv', index=False)
print(" Saved: profile_clean.csv")

transcript_clean.to_csv('cleaned_data/transcript_clean.csv', index=False)
print(" Saved: transcript_clean.csv")

offers_received.to_csv('cleaned_data/offers_received.csv', index=False)
print(" Saved: offers_received.csv")

offers_viewed.to_csv('cleaned_data/offers_viewed.csv', index=False)
print(" Saved: offers_viewed.csv")

offers_completed.to_csv('cleaned_data/offers_completed.csv', index=False)
print(" Saved: offers_completed.csv")

transactions.to_csv('cleaned_data/transactions.csv', index=False)
print(" Saved: transactions.csv")

customer_summary.to_csv('cleaned_data/customer_summary.csv', index=False)
print(" Saved: customer_summary.csv")

print(f"\n All datasets saved to 'cleaned_data/' folder")

# List saved files
print("\n Saved files:")
!ls -lh cleaned_data/

Create Base modelling dataset

In [ ]:
"""
Create the base modeling dataset - one row per offer instance
"""

print("=" * 80)
print(" CREATING BASE MODELING DATASET")
print("=" * 80)

# Start with offers_received as our base
# Each row = one decision moment (customer received an offer)
modeling_data = offers_received.copy()

print(f"\n Base dataset created:")
print(f"   Total rows: {len(modeling_data):,}")
print(f"   Columns: {modeling_data.shape[1]}")

# Add customer demographic features
print("\n Merging customer demographics...")
modeling_data = modeling_data.merge(
    profile_clean[['customer_id', 'gender', 'age_clean', 'income_clean',
                   'member_tenure_days', 'became_member_on']],
    on='customer_id',
    how='left'
)

# Calculate tenure at the time of offer (not current tenure)
modeling_data['tenure_at_offer'] = (
    modeling_data['timestamp'] - modeling_data['became_member_on']
).dt.days

print(f" Demographics added")
print(f"   Current columns: {modeling_data.shape[1]}")

# Display sample
print("\n Sample of base modeling data:")
display(modeling_data[['customer_id', 'timestamp', 'offer_type_simple',
                       'gender', 'age_clean', 'income_clean', 'tenure_at_offer']].head())

Helper Function for Time-Based Features

In [ ]:
"""
Create helper functions for time-based feature calculation
"""

print("=" * 80)
print("  CREATING HELPER FUNCTIONS")
print("=" * 80)

def calculate_rfm_features(customer_id, decision_time, transactions_df, lookback_days=None):
    """
    Calculate RFM (Recency, Frequency, Monetary) features for a customer
    using only transactions BEFORE the decision time.

    Parameters:
    -----------
    customer_id : str
        Customer identifier
    decision_time : datetime
        The decision moment (offer received time)
    transactions_df : DataFrame
        All transactions data
    lookback_days : int or None
        If specified, only look back this many days. If None, use all history.

    Returns:
    --------
    dict : Dictionary of RFM features
    """

    # Filter transactions for this customer BEFORE decision time
    customer_txns = transactions_df[
        (transactions_df['customer_id'] == customer_id) &
        (transactions_df['timestamp'] < decision_time)
    ].copy()

    # If lookback_days specified, further filter
    if lookback_days is not None:
        cutoff_time = decision_time - pd.Timedelta(days=lookback_days)
        customer_txns = customer_txns[customer_txns['timestamp'] >= cutoff_time]

    # Calculate features
    if len(customer_txns) == 0:
        return {
            'recency_days': np.nan,
            'frequency': 0,
            'monetary_total': 0,
            'monetary_mean': 0,
            'monetary_max': 0,
            'monetary_std': 0,
        }

    # Recency: days since last transaction
    last_txn_time = customer_txns['timestamp'].max()
    recency_days = (decision_time - last_txn_time).total_seconds() / 86400  # Convert to days

    # Frequency: number of transactions
    frequency = len(customer_txns)

    # Monetary: spending statistics
    monetary_total = customer_txns['amount'].sum()
    monetary_mean = customer_txns['amount'].mean()
    monetary_max = customer_txns['amount'].max()
    monetary_std = customer_txns['amount'].std() if len(customer_txns) > 1 else 0

    return {
        'recency_days': recency_days,
        'frequency': frequency,
        'monetary_total': monetary_total,
        'monetary_mean': monetary_mean,
        'monetary_max': monetary_max,
        'monetary_std': monetary_std,
    }


def calculate_offer_history_features(customer_id, decision_time,
                                     offers_received_df, offers_viewed_df,
                                     offers_completed_df):
    """
    Calculate offer interaction history features for a customer
    using only data BEFORE the decision time.

    Parameters:
    -----------
    customer_id : str
        Customer identifier
    decision_time : datetime
        The decision moment (offer received time)
    offers_received_df : DataFrame
        All offers received data
    offers_viewed_df : DataFrame
        All offers viewed data
    offers_completed_df : DataFrame
        All offers completed data

    Returns:
    --------
    dict : Dictionary of offer history features
    """

    # Filter offers BEFORE decision time
    prev_received = offers_received_df[
        (offers_received_df['customer_id'] == customer_id) &
        (offers_received_df['timestamp'] < decision_time)
    ]

    prev_viewed = offers_viewed_df[
        (offers_viewed_df['customer_id'] == customer_id) &
        (offers_viewed_df['timestamp'] < decision_time)
    ]

    prev_completed = offers_completed_df[
        (offers_completed_df['customer_id'] == customer_id) &
        (offers_completed_df['timestamp'] < decision_time)
    ]

    # Calculate counts
    num_prev_received = len(prev_received)
    num_prev_viewed = len(prev_viewed)
    num_prev_completed = len(prev_completed)

    # Calculate rates
    view_rate = num_prev_viewed / num_prev_received if num_prev_received > 0 else 0
    completion_rate = num_prev_completed / num_prev_received if num_prev_received > 0 else 0
    completion_given_view_rate = num_prev_completed / num_prev_viewed if num_prev_viewed > 0 else 0

    return {
        'prev_offers_received': num_prev_received,
        'prev_offers_viewed': num_prev_viewed,
        'prev_offers_completed': num_prev_completed,
        'prev_view_rate': view_rate,
        'prev_completion_rate': completion_rate,
        'prev_completion_given_view_rate': completion_given_view_rate,
    }

print(" Helper functions created:")
print("   • calculate_rfm_features()")
print("   • calculate_offer_history_features()")

Calculate RFM Features

In [ ]:
"""
Calculate RFM features for each offer instance
This is computationally intensive - we'll use vectorized operations where possible
"""

print("=" * 80)
print(" CALCULATING RFM FEATURES")
print("=" * 80)

from tqdm import tqdm
tqdm.pandas()

print("\n This may take a few minutes for 76K+ offer instances...")
print("   Processing RFM features for different time windows...\n")

# We'll calculate RFM for different lookback windows
# This gives us both recent and long-term behavior

# Initialize feature columns
rfm_windows = {
    'lifetime': None,  # All history
    '90d': 90,         # Last 90 days
    '30d': 30,         # Last 30 days
    '7d': 7,           # Last 7 days
}

for window_name, lookback_days in rfm_windows.items():
    print(f" Calculating RFM features for {window_name} window...")

    rfm_features = modeling_data.progress_apply(
        lambda row: calculate_rfm_features(
            row['customer_id'],
            row['timestamp'],
            transactions,
            lookback_days
        ),
        axis=1
    )

    # Convert to DataFrame and add to modeling_data
    rfm_df = pd.DataFrame(rfm_features.tolist())

    # Rename columns with window prefix
    rfm_df.columns = [f'rfm_{window_name}_{col}' for col in rfm_df.columns]

    # Add to modeling_data
    modeling_data = pd.concat([modeling_data, rfm_df], axis=1)

    print(f"    {window_name} features added")

print(f"\n All RFM features calculated!")
print(f"   Total columns now: {modeling_data.shape[1]}")

# Show sample of RFM features
print("\n Sample of RFM features:")
rfm_cols = [col for col in modeling_data.columns if 'rfm_' in col]
display(modeling_data[['customer_id', 'timestamp'] + rfm_cols[:8]].head())

Calculate Offer History Features

In [ ]:
"""
Calculate offer interaction history features
"""

print("=" * 80)
print(" CALCULATING OFFER HISTORY FEATURES")
print("=" * 80)

print("\n Processing offer interaction history...\n")

offer_history_features = modeling_data.progress_apply(
    lambda row: calculate_offer_history_features(
        row['customer_id'],
        row['timestamp'],
        offers_received,
        offers_viewed,
        offers_completed
    ),
    axis=1
)

# Convert to DataFrame
offer_history_df = pd.DataFrame(offer_history_features.tolist())

# Add to modeling_data
modeling_data = pd.concat([modeling_data, offer_history_df], axis=1)

print(f"\n Offer history features calculated!")
print(f"   Total columns now: {modeling_data.shape[1]}")

# Show sample
print("\n Sample of offer history features:")
offer_hist_cols = [col for col in modeling_data.columns if 'prev_' in col]
display(modeling_data[['customer_id', 'timestamp'] + offer_hist_cols].head(10))

# Show statistics
print("\n Offer history feature statistics:")
print(modeling_data[offer_hist_cols].describe())

Calculate Time-Based Features

In [ ]:
"""
Create time-based features from the offer timestamp
"""

print("=" * 80)
print(" CREATING TIME-BASED FEATURES")
print("=" * 80)

# Extract temporal features from timestamp
modeling_data['hour_of_day'] = modeling_data['timestamp'].dt.hour
modeling_data['day_of_week'] = modeling_data['timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
modeling_data['day_of_month'] = modeling_data['timestamp'].dt.day
modeling_data['month'] = modeling_data['timestamp'].dt.month
modeling_data['is_weekend'] = modeling_data['day_of_week'].isin([5, 6]).astype(int)
modeling_data['is_month_start'] = (modeling_data['day_of_month'] <= 7).astype(int)
modeling_data['is_month_end'] = (modeling_data['day_of_month'] >= 24).astype(int)

# Days since experiment start
START_DATE = pd.to_datetime('2018-01-01')
modeling_data['days_since_experiment_start'] = (
    modeling_data['timestamp'] - START_DATE
).dt.days

print(" Time-based features created:")
time_features = ['hour_of_day', 'day_of_week', 'day_of_month', 'month',
                 'is_weekend', 'is_month_start', 'is_month_end',
                 'days_since_experiment_start']
for feat in time_features:
    print(f"   • {feat}")

print(f"\n   Total columns now: {modeling_data.shape[1]}")

# Show sample
print("\n Sample of time-based features:")
display(modeling_data[['customer_id', 'timestamp'] + time_features].head(10))

In [ ]:
"""
CREATE A CLEAN COPY OF MODELING_DATA WITH NO INDEX ISSUES
This is our checkpoint - we'll work with this clean copy going forward
"""

print("=" * 80)
print(" CREATING CLEAN COPY OF MODELING_DATA")
print("=" * 80)

print(f"\n Original modeling_data:")
print(f"   Shape: {modeling_data.shape}")
print(f"   Has duplicate indices: {modeling_data.index.duplicated().any()}")

# Simple method: just reset index and copy
print("\n  Creating clean copy...")

modeling_data_clean = modeling_data.copy()
modeling_data_clean.reset_index(drop=True, inplace=True)

# Replace original
modeling_data = modeling_data_clean

print(f"\n Clean copy created:")
print(f"   Shape: {modeling_data.shape}")
print(f"   Has duplicate indices: {modeling_data.index.duplicated().any()}")
print(f"   Index is unique: {modeling_data.index.is_unique}")

# Fix all other dataframes
print("\n Fixing indices in other dataframes...")

# Fix transactions
print("   • transactions...")
transactions = transactions.copy()
transactions.reset_index(drop=True, inplace=True)
print(f"     ✓ Fixed. Has duplicates: {transactions.index.duplicated().any()}")

# Fix offers_completed
print("   • offers_completed...")
offers_completed = offers_completed.copy()
offers_completed.reset_index(drop=True, inplace=True)
print(f"     ✓ Fixed. Has duplicates: {offers_completed.index.duplicated().any()}")

# Fix offers_viewed
print("   • offers_viewed...")
offers_viewed = offers_viewed.copy()
offers_viewed.reset_index(drop=True, inplace=True)
print(f"     ✓ Fixed. Has duplicates: {offers_viewed.index.duplicated().any()}")

# Fix offers_received
print("   • offers_received...")
offers_received = offers_received.copy()
offers_received.reset_index(drop=True, inplace=True)
print(f"     ✓ Fixed. Has duplicates: {offers_received.index.duplicated().any()}")

print("\n ALL DATAFRAMES ARE NOW CLEAN AND INDEX-FREE!")

# Verify
print("\n Final verification:")
print(f"   modeling_data: {modeling_data.shape}, unique index: {modeling_data.index.is_unique}")
print(f"   transactions: {transactions.shape}, unique index: {transactions.index.is_unique}")
print(f"   offers_completed: {offers_completed.shape}, unique index: {offers_completed.index.is_unique}")
print(f"   offers_viewed: {offers_viewed.shape}, unique index: {offers_viewed.index.is_unique}")
print(f"   offers_received: {offers_received.shape}, unique index: {offers_received.index.is_unique}")

print("\n Ready to calculate outcomes!")

Create helper functions

In [ ]:
"""
COMPLETELY NEW APPROACH - Simple iteration without any fancy pandas operations
"""

print("=" * 80)
print(" CREATING OUTCOME CALCULATION FUNCTIONS (NEW SIMPLE APPROACH)")
print("=" * 80)

def calculate_conversion_outcome_simple(customer_id, decision_time, offer_id,
                                        window_days, transactions_df,
                                        offers_completed_df):
    """
    Simple approach - just loop through dataframes
    """
    window_end = decision_time + pd.Timedelta(days=window_days)

    # Check column name
    offer_col = 'offer_id' if 'offer_id' in offers_completed_df.columns else 'offer id'

    # Count completed offers by looping
    num_completed = 0
    for idx in range(len(offers_completed_df)):
        row = offers_completed_df.iloc[idx]
        if (row['customer_id'] == customer_id and
            str(row[offer_col]) == str(offer_id) and
            row['timestamp'] >= decision_time and
            row['timestamp'] <= window_end):
            num_completed += 1

    # Count transactions by looping
    num_txns = 0
    total_spent = 0.0
    for idx in range(len(transactions_df)):
        row = transactions_df.iloc[idx]
        if (row['customer_id'] == customer_id and
            row['timestamp'] >= decision_time and
            row['timestamp'] <= window_end):
            num_txns += 1
            total_spent += float(row['amount'])

    # Calculate outcomes
    offer_completed = 1 if num_completed > 0 else 0
    any_transaction = 1 if num_txns > 0 else 0
    conversion = 1 if (offer_completed or any_transaction) else 0

    return {
        'offer_completed': offer_completed,
        'any_transaction': any_transaction,
        'conversion': conversion,
        'num_transactions_in_window': num_txns,
        'total_spent_in_window': total_spent,
    }


def calculate_retention_outcome_simple(customer_id, decision_time, window_days,
                                       transactions_df):
    """
    Simple approach - just loop through dataframe
    """
    window_end = decision_time + pd.Timedelta(days=window_days)

    # Count transactions by looping
    num_txns = 0
    total_spent = 0.0
    for idx in range(len(transactions_df)):
        row = transactions_df.iloc[idx]
        if (row['customer_id'] == customer_id and
            row['timestamp'] >= decision_time and
            row['timestamp'] <= window_end):
            num_txns += 1
            total_spent += float(row['amount'])

    # Calculate outcomes
    is_active = 1 if num_txns > 0 else 0
    avg_spent = total_spent / num_txns if num_txns > 0 else 0.0

    return {
        'retention_active': is_active,
        'retention_num_transactions': num_txns,
        'retention_total_spent': total_spent,
        'retention_avg_spent': avg_spent,
    }

print(" Simple outcome calculation functions created!")
print("   (These use basic loops - slower but 100% reliable)")

# Test
print("\n Testing functions...")
try:
    test_conv = calculate_conversion_outcome_simple(
        customer_id=modeling_data['customer_id'].iloc[0],
        decision_time=modeling_data['timestamp'].iloc[0],
        offer_id=modeling_data['offer_id'].iloc[0],
        window_days=14,
        transactions_df=transactions,
        offers_completed_df=offers_completed
    )
    print(f" Conversion test passed! Result: {test_conv}")

    test_ret = calculate_retention_outcome_simple(
        customer_id=modeling_data['customer_id'].iloc[0],
        decision_time=modeling_data['timestamp'].iloc[0],
        window_days=30,
        transactions_df=transactions
    )
    print(f" Retention test passed! Result: {test_ret}")

    print("\n BOTH TESTS PASSED! Functions are ready!")

except Exception as e:
    print(f" Test failed: {e}")
    import traceback
    traceback.print_exc()

Calculate conversion outcome

In [ ]:
"""
Calculate conversion outcomes - OPTIMIZED VERSION
Pre-filter dataframes by customer to make it much faster
"""

print("=" * 80)
print(" CALCULATING CONVERSION OUTCOMES (14-DAY WINDOW) - OPTIMIZED")
print("=" * 80)

print(f"\n Processing {len(modeling_data):,} offer instances...")
print("   Using optimized approach - should take ~3-5 minutes\n")

# Pre-group data by customer_id for faster lookup
print(" Pre-grouping data by customer...")
transactions_by_customer = transactions.groupby('customer_id')
offers_completed_by_customer = offers_completed.groupby('customer_id')

# Check column name
offer_col = 'offer_id' if 'offer_id' in offers_completed.columns else 'offer id'

print(" Pre-grouping complete! Starting calculations...\n")

# Lists to store results
results_offer_completed = []
results_any_transaction = []
results_conversion = []
results_num_transactions = []
results_total_spent = []

# Process each row
from tqdm import tqdm

for i in tqdm(range(len(modeling_data)), desc="Conversion outcomes"):
    customer_id = modeling_data['customer_id'].iloc[i]
    decision_time = modeling_data['timestamp'].iloc[i]
    offer_id = str(modeling_data['offer_id'].iloc[i])
    window_end = decision_time + pd.Timedelta(days=CONVERSION_WINDOW_DAYS)

    # Get this customer's data only
    try:
        customer_offers = offers_completed_by_customer.get_group(customer_id)
    except KeyError:
        customer_offers = pd.DataFrame()

    try:
        customer_txns = transactions_by_customer.get_group(customer_id)
    except KeyError:
        customer_txns = pd.DataFrame()

    # Count completed offers (much smaller dataset now)
    num_completed = 0
    if len(customer_offers) > 0:
        for idx in range(len(customer_offers)):
            row = customer_offers.iloc[idx]
            if (str(row[offer_col]) == offer_id and
                row['timestamp'] >= decision_time and
                row['timestamp'] <= window_end):
                num_completed += 1

    # Count transactions (much smaller dataset now)
    num_txns = 0
    total_spent = 0.0
    if len(customer_txns) > 0:
        for idx in range(len(customer_txns)):
            row = customer_txns.iloc[idx]
            if (row['timestamp'] >= decision_time and
                row['timestamp'] <= window_end):
                num_txns += 1
                total_spent += float(row['amount'])

    # Calculate outcomes
    offer_completed = 1 if num_completed > 0 else 0
    any_transaction = 1 if num_txns > 0 else 0
    conversion = 1 if (offer_completed or any_transaction) else 0

    results_offer_completed.append(offer_completed)
    results_any_transaction.append(any_transaction)
    results_conversion.append(conversion)
    results_num_transactions.append(num_txns)
    results_total_spent.append(total_spent)

# Add results as new columns
modeling_data['conv_14d_offer_completed'] = results_offer_completed
modeling_data['conv_14d_any_transaction'] = results_any_transaction
modeling_data['conv_14d_conversion'] = results_conversion
modeling_data['conv_14d_num_transactions_in_window'] = results_num_transactions
modeling_data['conv_14d_total_spent_in_window'] = results_total_spent

print(f"\n Conversion outcomes calculated!")
print(f"   Total columns now: {modeling_data.shape[1]}")

# Statistics
print("\n Conversion Outcome Statistics:")
conv_cols = ['conv_14d_offer_completed', 'conv_14d_any_transaction',
             'conv_14d_conversion', 'conv_14d_num_transactions_in_window',
             'conv_14d_total_spent_in_window']
print(modeling_data[conv_cols].describe())

# Conversion rate
conv_rate = modeling_data['conv_14d_conversion'].mean() * 100
print(f"\n Overall 14-day conversion rate: {conv_rate:.2f}%")

# Sample
print("\n Sample of conversion outcomes:")
display(modeling_data[['customer_id', 'timestamp', 'offer_type_simple'] + conv_cols].head(10))

Calculate Retention Outcomes

In [ ]:
"""
Calculate retention outcomes - OPTIMIZED VERSION
"""

print("=" * 80)
print(" CALCULATING RETENTION OUTCOMES (30-DAY WINDOW) - OPTIMIZED")
print("=" * 80)

print(f"\n Processing {len(modeling_data):,} offer instances...")
print("   Using optimized approach - should take ~2-3 minutes\n")

# Use the same pre-grouped data
# transactions_by_customer is already created in previous cell

# Lists to store results
results_active = []
results_num_transactions = []
results_total_spent = []
results_avg_spent = []

# Process each row
from tqdm import tqdm

for i in tqdm(range(len(modeling_data)), desc="Retention outcomes"):
    customer_id = modeling_data['customer_id'].iloc[i]
    decision_time = modeling_data['timestamp'].iloc[i]
    window_end = decision_time + pd.Timedelta(days=RETENTION_WINDOW_DAYS)

    # Get this customer's transactions only
    try:
        customer_txns = transactions_by_customer.get_group(customer_id)
    except KeyError:
        customer_txns = pd.DataFrame()

    # Count transactions
    num_txns = 0
    total_spent = 0.0
    if len(customer_txns) > 0:
        for idx in range(len(customer_txns)):
            row = customer_txns.iloc[idx]
            if (row['timestamp'] >= decision_time and
                row['timestamp'] <= window_end):
                num_txns += 1
                total_spent += float(row['amount'])

    # Calculate outcomes
    is_active = 1 if num_txns > 0 else 0
    avg_spent = total_spent / num_txns if num_txns > 0 else 0.0

    results_active.append(is_active)
    results_num_transactions.append(num_txns)
    results_total_spent.append(total_spent)
    results_avg_spent.append(avg_spent)

# Add results as new columns
modeling_data['ret_30d_retention_active'] = results_active
modeling_data['ret_30d_retention_num_transactions'] = results_num_transactions
modeling_data['ret_30d_retention_total_spent'] = results_total_spent
modeling_data['ret_30d_retention_avg_spent'] = results_avg_spent

print(f"\n Retention outcomes calculated!")
print(f"   Total columns now: {modeling_data.shape[1]}")

# Statistics
print("\n Retention Outcome Statistics:")
ret_cols = ['ret_30d_retention_active', 'ret_30d_retention_num_transactions',
            'ret_30d_retention_total_spent', 'ret_30d_retention_avg_spent']
print(modeling_data[ret_cols].describe())

# Retention rate
ret_rate = modeling_data['ret_30d_retention_active'].mean() * 100
print(f"\n Overall 30-day retention rate: {ret_rate:.2f}%")

# Sample
print("\n Sample of retention outcomes:")
display(modeling_data[['customer_id', 'timestamp', 'offer_type_simple'] + ret_cols].head(10))

Analyze Outcomes by Offer Type

In [ ]:
"""
Analyze conversion and retention by offer type
"""

print("=" * 80)
print(" OUTCOME ANALYSIS BY OFFER TYPE")
print("=" * 80)

# Group by offer type
outcome_by_offer = modeling_data.groupby('offer_type_simple').agg({
    'conv_14d_conversion': ['count', 'mean'],
    'conv_14d_offer_completed': 'mean',
    'conv_14d_any_transaction': 'mean',
    'conv_14d_total_spent_in_window': 'mean',
    'ret_30d_retention_active': 'mean',
    'ret_30d_retention_num_transactions': 'mean',
    'ret_30d_retention_total_spent': 'mean',
}).round(4)

outcome_by_offer.columns = ['_'.join(col).strip() for col in outcome_by_offer.columns.values]
outcome_by_offer = outcome_by_offer.rename(columns={
    'conv_14d_conversion_count': 'num_instances',
    'conv_14d_conversion_mean': 'conversion_rate',
    'conv_14d_offer_completed_mean': 'offer_completion_rate',
    'conv_14d_any_transaction_mean': 'transaction_rate',
    'conv_14d_total_spent_in_window_mean': 'avg_revenue_14d',
    'ret_30d_retention_active_mean': 'retention_rate',
    'ret_30d_retention_num_transactions_mean': 'avg_transactions_30d',
    'ret_30d_retention_total_spent_mean': 'avg_revenue_30d',
})

print("\n Outcomes by Offer Type:")
display(outcome_by_offer)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Conversion rate by offer type
outcome_by_offer['conversion_rate'].plot(
    kind='bar', ax=axes[0, 0], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[0, 0].set_title('Conversion Rate by Offer Type', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Conversion Rate')
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].axhline(modeling_data['conv_14d_conversion'].mean(), color='red',
                   linestyle='--', label='Overall Avg')
axes[0, 0].legend()

# Offer completion rate by offer type
outcome_by_offer['offer_completion_rate'].plot(
    kind='bar', ax=axes[0, 1], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[0, 1].set_title('Offer Completion Rate by Type', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Completion Rate')
axes[0, 1].set_xlabel('')
axes[0, 1].tick_params(axis='x', rotation=0)
axes[0, 1].grid(axis='y', alpha=0.3)

# Average revenue (14d) by offer type
outcome_by_offer['avg_revenue_14d'].plot(
    kind='bar', ax=axes[0, 2], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[0, 2].set_title('Avg Revenue (14d) by Offer Type', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Average Revenue ($)')
axes[0, 2].set_xlabel('')
axes[0, 2].tick_params(axis='x', rotation=0)
axes[0, 2].grid(axis='y', alpha=0.3)

# Retention rate by offer type
outcome_by_offer['retention_rate'].plot(
    kind='bar', ax=axes[1, 0], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[1, 0].set_title('Retention Rate (30d) by Offer Type', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Retention Rate')
axes[1, 0].set_xlabel('')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].axhline(modeling_data['ret_30d_retention_active'].mean(), color='red',
                   linestyle='--', label='Overall Avg')
axes[1, 0].legend()

# Avg transactions (30d) by offer type
outcome_by_offer['avg_transactions_30d'].plot(
    kind='bar', ax=axes[1, 1], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[1, 1].set_title('Avg Transactions (30d) by Offer Type', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Avg # Transactions')
axes[1, 1].set_xlabel('')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].grid(axis='y', alpha=0.3)

# Avg revenue (30d) by offer type
outcome_by_offer['avg_revenue_30d'].plot(
    kind='bar', ax=axes[1, 2], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black'
)
axes[1, 2].set_title('Avg Revenue (30d) by Offer Type', fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel('Average Revenue ($)')
axes[1, 2].set_xlabel('')
axes[1, 2].tick_params(axis='x', rotation=0)
axes[1, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Outcome analysis complete!")

Create Additional Derived Features

In [ ]:
"""
Create additional derived features based on existing features
"""

print("=" * 80)
print("🔧 CREATING DERIVED FEATURES")
print("=" * 80)

# Customer value score (combines spending and frequency)
modeling_data['customer_value_score'] = (
    modeling_data['rfm_lifetime_monetary_total'] *
    np.log1p(modeling_data['rfm_lifetime_frequency'])
)

# Engagement score (combines offer interactions)
modeling_data['engagement_score'] = (
    modeling_data['prev_view_rate'] * 0.5 +
    modeling_data['prev_completion_rate'] * 0.5
)

# Recent activity flag (transacted in last 7 days)
modeling_data['is_recently_active'] = (
    modeling_data['rfm_7d_frequency'] > 0
).astype(int)

# High spender flag (above median spending)
median_lifetime_spend = modeling_data['rfm_lifetime_monetary_total'].median()
modeling_data['is_high_spender'] = (
    modeling_data['rfm_lifetime_monetary_total'] > median_lifetime_spend
).astype(int)

# Offer experience (number of previous offers received)
modeling_data['offer_experience_level'] = pd.cut(
    modeling_data['prev_offers_received'],
    bins=[-1, 0, 2, 5, 100],
    labels=['new', 'low', 'medium', 'high']
)

# Age group
modeling_data['age_group'] = pd.cut(
    modeling_data['age_clean'],
    bins=[0, 30, 45, 60, 100],
    labels=['young', 'middle', 'senior', 'elderly']
)

# Income group
modeling_data['income_group'] = pd.cut(
    modeling_data['income_clean'],
    bins=[0, 50000, 75000, 100000, 200000],
    labels=['low', 'medium', 'high', 'very_high']
)

print(" Derived features created:")
derived_features = ['customer_value_score', 'engagement_score', 'is_recently_active',
                    'is_high_spender', 'offer_experience_level', 'age_group', 'income_group']
for feat in derived_features:
    print(f"   • {feat}")

print(f"\n   Total columns now: {modeling_data.shape[1]}")

# Show sample
print("\n Sample of derived features:")
display(modeling_data[['customer_id'] + derived_features].head(10))

# Show statistics
print("\n Derived feature statistics:")
print(modeling_data[['customer_value_score', 'engagement_score']].describe())

Final Data Quality Check and Save

In [ ]:
"""
Final data quality check and save the modeling dataset
"""

print("=" * 80)
print("🔍 FINAL DATA QUALITY CHECK")
print("=" * 80)

print(f"\n Final Dataset Shape: {modeling_data.shape}")
print(f"   Rows: {modeling_data.shape[0]:,}")
print(f"   Columns: {modeling_data.shape[1]}")

# Check for missing values
print("\n Missing Values Check:")
missing_summary = modeling_data.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

if len(missing_summary) > 0:
    print(f"\n⚠️  Columns with missing values:")
    print(missing_summary.head(20))
    print(f"\n   Missing percentage:")
    print((missing_summary / len(modeling_data) * 100).round(2).head(20))
else:
    print(" No missing values!")

# Check target variable distribution
print("\n🎯 Target Variable Distribution:")
print("\nConversion (14d):")
print(modeling_data['conv_14d_conversion'].value_counts())
print(f"Conversion rate: {modeling_data['conv_14d_conversion'].mean()*100:.2f}%")

print("\nRetention (30d):")
print(modeling_data['ret_30d_retention_active'].value_counts())
print(f"Retention rate: {modeling_data['ret_30d_retention_active'].mean()*100:.2f}%")

# Memory usage
print(f"\n Memory Usage: {modeling_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Save the modeling dataset
print("\n SAVING MODELING DATASET")
print("=" * 80)

modeling_data.to_csv('cleaned_data/modeling_dataset_complete.csv', index=False)
print(" Saved: modeling_dataset_complete.csv")

# Also save a pickle file for faster loading
modeling_data.to_pickle('cleaned_data/modeling_dataset_complete.pkl')
print(" Saved: modeling_dataset_complete.pkl")

print(f"\n Modeling dataset saved successfully!")
print(f"   Location: cleaned_data/")
print(f"   Files: modeling_dataset_complete.csv and .pkl")

Select features for clustering

In [ ]:
"""
Select and prepare features for clustering
"""

print("=" * 80)
print(" SELECTING FEATURES FOR CLUSTERING")
print("=" * 80)

# Select clustering features - focusing on customer behavior
clustering_features = [
    # RFM features (most important)
    'rfm_lifetime_recency_days',
    'rfm_lifetime_frequency',
    'rfm_lifetime_monetary_total',
    'rfm_lifetime_monetary_mean',

    # Recent behavior
    'rfm_30d_frequency',
    'rfm_30d_monetary_total',

    # Offer engagement
    'prev_offers_received',
    'prev_view_rate',
    'prev_completion_rate',

    # Demographics
    'age_clean',
    'income_clean',
    'member_tenure_days',
]

print(f"\n Selected {len(clustering_features)} features for clustering:")
for i, feat in enumerate(clustering_features, 1):
    print(f"   {i}. {feat}")

# Create clustering dataset (one row per customer, not per offer)
print("\n Aggregating to customer level...")

# Get unique customers with their average characteristics
customer_level_data = modeling_data.groupby('customer_id').agg({
    'rfm_lifetime_recency_days': 'first',  # These are constant per customer
    'rfm_lifetime_frequency': 'first',
    'rfm_lifetime_monetary_total': 'first',
    'rfm_lifetime_monetary_mean': 'first',
    'rfm_30d_frequency': 'first',
    'rfm_30d_monetary_total': 'first',
    'prev_offers_received': 'first',
    'prev_view_rate': 'first',
    'prev_completion_rate': 'first',
    'age_clean': 'first',
    'income_clean': 'first',
    'member_tenure_days': 'first',
    'gender': 'first',
    'offer_type_simple': 'first',  # Keep for analysis
    'conv_14d_conversion': 'mean',  # Average conversion rate
    'ret_30d_retention_active': 'mean',  # Average retention
}).reset_index()

print(f" Customer-level dataset created:")
print(f"   Shape: {customer_level_data.shape}")
print(f"   Unique customers: {len(customer_level_data):,}")

# Handle missing values
print("\n Handling missing values...")
print(f"Missing values before:")
print(customer_level_data[clustering_features].isnull().sum())

# Fill missing values with median
for col in clustering_features:
    if customer_level_data[col].isnull().any():
        median_val = customer_level_data[col].median()
        customer_level_data[col].fillna(median_val, inplace=True)

print(f"\n Missing values after:")
print(customer_level_data[clustering_features].isnull().sum())

# Show sample
print("\n Sample of customer-level data:")
display(customer_level_data[['customer_id'] + clustering_features[:6]].head(10))

Scale Features

In [ ]:
"""
Scale features for clustering
"""

print("=" * 80)
print("  SCALING FEATURES")
print("=" * 80)

from sklearn.preprocessing import StandardScaler

# Extract feature matrix
X = customer_level_data[clustering_features].values

print(f"\n Feature matrix shape: {X.shape}")
print(f"   Samples (customers): {X.shape[0]:,}")
print(f"   Features: {X.shape[1]}")

# Show feature statistics before scaling
print("\n Feature statistics BEFORE scaling:")
print(pd.DataFrame(X, columns=clustering_features).describe())

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n Features scaled using StandardScaler (mean=0, std=1)")

# Show feature statistics after scaling
print("\n Feature statistics AFTER scaling:")
print(pd.DataFrame(X_scaled, columns=clustering_features).describe())

# Save scaled data
customer_level_data_scaled = customer_level_data.copy()
for i, col in enumerate(clustering_features):
    customer_level_data_scaled[f'{col}_scaled'] = X_scaled[:, i]

print(f"\n Scaled features ready for clustering!")

Determine Optimal Number of Clusters

In [ ]:
"""
Determine optimal number of clusters using Elbow Method and Silhouette Score
"""

print("=" * 80)
print(" DETERMINING OPTIMAL NUMBER OF CLUSTERS")
print("=" * 80)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

print("\n Testing different numbers of clusters (2-10)...")
print("   This will take a minute...\n")

# Test different numbers of clusters
k_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in k_range:
    print(f"Testing k={k}...", end=" ")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(silhouette)
    print(f"Inertia: {kmeans.inertia_:.2f}, Silhouette: {silhouette:.3f}")

print("\n Testing complete!")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow plot
axes[0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(k_range)

# Silhouette plot
axes[1].plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(k_range)

plt.tight_layout()
plt.show()

# Find best k (highest silhouette score)
best_k_idx = np.argmax(silhouette_scores)
best_k = k_range[best_k_idx]

print(f"\n RESULTS:")
print(f"   Best k (by Silhouette Score): {best_k}")
print(f"   Silhouette Score: {silhouette_scores[best_k_idx]:.3f}")

print("\n RECOMMENDATION:")
print(f"   Based on the analysis, k={best_k} clusters is optimal.")
print(f"   However, for interpretability, we'll use k=4 or k=5")
print(f"   (Common practice: 3-5 segments are easier to act on)")

# Let's use k=4 for good interpretability
optimal_k = 4
print(f"\n We'll proceed with k={optimal_k} clusters")

Apply K-Means Clustering

In [ ]:
"""
Apply K-Means clustering with optimal k
"""

print("=" * 80)
print(" APPLYING K-MEANS CLUSTERING")
print("=" * 80)

optimal_k = 4  # Can adjust this based on previous cell results

print(f"\n Fitting K-Means with k={optimal_k} clusters...")

# Fit K-Means
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=500)
customer_level_data['segment'] = kmeans_final.fit_predict(X_scaled)

print(f" Clustering complete!")

# Calculate metrics
inertia = kmeans_final.inertia_
silhouette = silhouette_score(X_scaled, customer_level_data['segment'])

print(f"\n Clustering Metrics:")
print(f"   Inertia: {inertia:.2f}")
print(f"   Silhouette Score: {silhouette:.3f}")

# Segment sizes
print(f"\n Segment Distribution:")
segment_counts = customer_level_data['segment'].value_counts().sort_index()
for seg, count in segment_counts.items():
    pct = (count / len(customer_level_data)) * 100
    print(f"   Segment {seg}: {count:,} customers ({pct:.1f}%)")

# Visualize segment distribution
fig, ax = plt.subplots(figsize=(10, 6))
segment_counts.plot(kind='bar', ax=ax, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#F7DC6F'],
                    edgecolor='black')
ax.set_title('Customer Segment Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Segment', fontsize=12)
ax.set_ylabel('Number of Customers', fontsize=12)
ax.tick_params(axis='x', rotation=0)
ax.grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, v in enumerate(segment_counts):
    ax.text(i, v + 50, f'{v:,}\n({v/len(customer_level_data)*100:.1f}%)',
            ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Segments created successfully!")

Segment Profiling

In [ ]:
"""
Profile each segment - calculate average characteristics
"""

print("=" * 80)
print(" SEGMENT PROFILING")
print("=" * 80)

# Calculate segment profiles
segment_profiles = customer_level_data.groupby('segment').agg({
    # RFM
    'rfm_lifetime_recency_days': 'mean',
    'rfm_lifetime_frequency': 'mean',
    'rfm_lifetime_monetary_total': 'mean',
    'rfm_lifetime_monetary_mean': 'mean',

    # Recent behavior
    'rfm_30d_frequency': 'mean',
    'rfm_30d_monetary_total': 'mean',

    # Engagement
    'prev_offers_received': 'mean',
    'prev_view_rate': 'mean',
    'prev_completion_rate': 'mean',

    # Demographics
    'age_clean': 'mean',
    'income_clean': 'mean',
    'member_tenure_days': 'mean',

    # Outcomes
    'conv_14d_conversion': 'mean',
    'ret_30d_retention_active': 'mean',

    # Count
    'customer_id': 'count'
}).round(2)

segment_profiles = segment_profiles.rename(columns={'customer_id': 'customer_count'})

print("\n SEGMENT PROFILES:")
print("=" * 80)
display(segment_profiles)

# Transpose for better readability
print("\n SEGMENT PROFILES (Transposed):")
print("=" * 80)
display(segment_profiles.T)

In [ ]:
"""
Visualize key segment characteristics
"""

print("=" * 80)
print(" VISUALIZING SEGMENT CHARACTERISTICS")
print("=" * 80)

fig, axes = plt.subplots(3, 3, figsize=(16, 14))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#F7DC6F']

# 1. Average Frequency
segment_profiles['rfm_lifetime_frequency'].plot(
    kind='bar', ax=axes[0, 0], color=colors, edgecolor='black'
)
axes[0, 0].set_title('Average Purchase Frequency', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Avg Transactions')
axes[0, 0].set_xlabel('Segment')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Average Total Spend
segment_profiles['rfm_lifetime_monetary_total'].plot(
    kind='bar', ax=axes[0, 1], color=colors, edgecolor='black'
)
axes[0, 1].set_title('Average Total Spend (Lifetime)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Avg Spend ($)')
axes[0, 1].set_xlabel('Segment')
axes[0, 1].tick_params(axis='x', rotation=0)
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Average Recency
segment_profiles['rfm_lifetime_recency_days'].plot(
    kind='bar', ax=axes[0, 2], color=colors, edgecolor='black'
)
axes[0, 2].set_title('Average Recency (Days Since Last Purchase)', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Days')
axes[0, 2].set_xlabel('Segment')
axes[0, 2].tick_params(axis='x', rotation=0)
axes[0, 2].grid(axis='y', alpha=0.3)

# 4. Recent Activity (30d frequency)
segment_profiles['rfm_30d_frequency'].plot(
    kind='bar', ax=axes[1, 0], color=colors, edgecolor='black'
)
axes[1, 0].set_title('Recent Activity (30d Transactions)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Avg Transactions')
axes[1, 0].set_xlabel('Segment')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)

# 5. Offer Engagement (view rate)
segment_profiles['prev_view_rate'].plot(
    kind='bar', ax=axes[1, 1], color=colors, edgecolor='black'
)
axes[1, 1].set_title('Offer View Rate', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('View Rate')
axes[1, 1].set_xlabel('Segment')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].grid(axis='y', alpha=0.3)

# 6. Offer Completion Rate
segment_profiles['prev_completion_rate'].plot(
    kind='bar', ax=axes[1, 2], color=colors, edgecolor='black'
)
axes[1, 2].set_title('Offer Completion Rate', fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel('Completion Rate')
axes[1, 2].set_xlabel('Segment')
axes[1, 2].tick_params(axis='x', rotation=0)
axes[1, 2].grid(axis='y', alpha=0.3)

# 7. Conversion Rate
segment_profiles['conv_14d_conversion'].plot(
    kind='bar', ax=axes[2, 0], color=colors, edgecolor='black'
)
axes[2, 0].set_title('Conversion Rate (14d)', fontsize=12, fontweight='bold')
axes[2, 0].set_ylabel('Conversion Rate')
axes[2, 0].set_xlabel('Segment')
axes[2, 0].tick_params(axis='x', rotation=0)
axes[2, 0].grid(axis='y', alpha=0.3)

# 8. Retention Rate
segment_profiles['ret_30d_retention_active'].plot(
    kind='bar', ax=axes[2, 1], color=colors, edgecolor='black'
)
axes[2, 1].set_title('Retention Rate (30d)', fontsize=12, fontweight='bold')
axes[2, 1].set_ylabel('Retention Rate')
axes[2, 1].set_xlabel('Segment')
axes[2, 1].tick_params(axis='x', rotation=0)
axes[2, 1].grid(axis='y', alpha=0.3)

# 9. Average Income
segment_profiles['income_clean'].plot(
    kind='bar', ax=axes[2, 2], color=colors, edgecolor='black'
)
axes[2, 2].set_title('Average Income', fontsize=12, fontweight='bold')
axes[2, 2].set_ylabel('Income ($)')
axes[2, 2].set_xlabel('Segment')
axes[2, 2].tick_params(axis='x', rotation=0)
axes[2, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Segment visualization complete!")

In [ ]:
"""
Assign meaningful names to segments based on their characteristics
"""

print("=" * 80)
print("  NAMING SEGMENTS")
print("=" * 80)

# Analyze each segment to assign names
print("\n Analyzing segment characteristics...")
print("=" * 80)

for seg in range(optimal_k):
    print(f"\n SEGMENT {seg}:")
    print("-" * 40)

    seg_data = segment_profiles.loc[seg]

    print(f"Size: {int(seg_data['customer_count']):,} customers")
    print(f"Frequency: {seg_data['rfm_lifetime_frequency']:.1f} transactions")
    print(f"Total Spend: ${seg_data['rfm_lifetime_monetary_total']:.2f}")
    print(f"Avg Transaction: ${seg_data['rfm_lifetime_monetary_mean']:.2f}")
    print(f"Recency: {seg_data['rfm_lifetime_recency_days']:.0f} days")
    print(f"Recent Activity (30d): {seg_data['rfm_30d_frequency']:.2f} txns")
    print(f"Offer View Rate: {seg_data['prev_view_rate']:.1%}")
    print(f"Offer Completion Rate: {seg_data['prev_completion_rate']:.1%}")
    print(f"Conversion Rate: {seg_data['conv_14d_conversion']:.1%}")
    print(f"Retention Rate: {seg_data['ret_30d_retention_active']:.1%}")
    print(f"Avg Income: ${seg_data['income_clean']:,.0f}")

# Based on the profiles, assign names
# You'll need to adjust these based on your actual results
segment_names = {
    0: "Segment 0",  # Will update based on characteristics
    1: "Segment 1",
    2: "Segment 2",
    3: "Segment 3"
}

# Let me create a mapping based on common patterns
# We'll determine this after seeing the results
print("\n" + "=" * 80)
print(" SEGMENT NAME SUGGESTIONS:")
print("=" * 80)
print("""
Based on typical RFM segmentation patterns, segments usually fall into:

1. "Champions" / "Loyal Regulars"
   - High frequency, high spend, low recency
   - Best customers, need retention focus

2. "Potential Loyalists" / "Promising"
   - Medium-high frequency, growing spend
   - Target for upselling and loyalty programs

3. "At Risk" / "Needs Attention"
   - Previously active, now dormant (high recency)
   - Target for win-back campaigns

4. "New/Low Value" / "Occasional"
   - Low frequency, low spend
   - Target for activation and engagement

Examine the profiles above and update the names accordingly.
""")

# For now, let's create descriptive labels based on spend and frequency
segment_descriptions = {}
for seg in range(optimal_k):
    freq = segment_profiles.loc[seg, 'rfm_lifetime_frequency']
    spend = segment_profiles.loc[seg, 'rfm_lifetime_monetary_total']
    recency = segment_profiles.loc[seg, 'rfm_lifetime_recency_days']

    if freq > segment_profiles['rfm_lifetime_frequency'].median() and spend > segment_profiles['rfm_lifetime_monetary_total'].median():
        if recency < segment_profiles['rfm_lifetime_recency_days'].median():
            label = f"Champions (Seg {seg})"
        else:
            label = f"At Risk High Value (Seg {seg})"
    elif freq > segment_profiles['rfm_lifetime_frequency'].median():
        label = f"Frequent Buyers (Seg {seg})"
    elif spend > segment_profiles['rfm_lifetime_monetary_total'].median():
        label = f"Big Spenders (Seg {seg})"
    else:
        label = f"Casual Shoppers (Seg {seg})"

    segment_descriptions[seg] = label

print("\n  AUTO-GENERATED SEGMENT LABELS:")
print("=" * 80)
for seg, label in segment_descriptions.items():
    print(f"Segment {seg}: {label}")

# Add segment names to customer data
customer_level_data['segment_name'] = customer_level_data['segment'].map(segment_descriptions)

print("\n Segments named!")

In [ ]:
"""
Analyze segment demographics
"""

print("=" * 80)
print(" SEGMENT DEMOGRAPHICS")
print("=" * 80)

# Gender distribution by segment
print("\n Gender Distribution by Segment:")
gender_dist = pd.crosstab(
    customer_level_data['segment_name'],
    customer_level_data['gender'],
    normalize='index'
) * 100

display(gender_dist.round(1))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gender distribution
gender_dist.plot(kind='bar', ax=axes[0], stacked=False,
                color=['#3498db', '#e74c3c', '#95a5a6'], edgecolor='black')
axes[0].set_title('Gender Distribution by Segment (%)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Percentage (%)')
axes[0].set_xlabel('Segment')
axes[0].legend(title='Gender')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Average age by segment
avg_age = customer_level_data.groupby('segment_name')['age_clean'].mean()
avg_age.plot(kind='bar', ax=axes[1], color='#9b59b6', edgecolor='black')
axes[1].set_title('Average Age by Segment', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Age (years)')
axes[1].set_xlabel('Segment')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Demographics analysis complete!")

In [ ]:
"""
Add segment labels back to the main modeling dataset
"""

print("=" * 80)
print(" ADDING SEGMENTS TO MAIN DATASET")
print("=" * 80)

# Merge segments back to modeling_data
modeling_data = modeling_data.merge(
    customer_level_data[['customer_id', 'segment', 'segment_name']],
    on='customer_id',
    how='left'
)

print(f" Segments added to modeling_data!")
print(f"   Shape: {modeling_data.shape}")

# Show sample
print("\n Sample with segments:")
display(modeling_data[['customer_id', 'segment', 'segment_name', 'offer_type_simple',
                       'conv_14d_conversion', 'ret_30d_retention_active']].head(10))

# Check distribution
print("\n Segment distribution in modeling_data:")
print(modeling_data['segment_name'].value_counts())

print("\n Segmentation complete and integrated!")

Segment Profitability Analysis

In [ ]:
"""
Analyze segment profitability and offer response
"""

print("=" * 80)
print(" SEGMENT PROFITABILITY ANALYSIS")
print("=" * 80)

# Calculate profitability metrics by segment
segment_profitability = modeling_data.groupby('segment_name').agg({
    # Revenue metrics
    'conv_14d_total_spent_in_window': ['mean', 'sum'],
    'ret_30d_retention_total_spent': ['mean', 'sum'],

    # Conversion metrics
    'conv_14d_conversion': 'mean',
    'conv_14d_offer_completed': 'mean',
    'ret_30d_retention_active': 'mean',

    # Engagement
    'prev_view_rate': 'mean',
    'prev_completion_rate': 'mean',

    # Count
    'customer_id': 'count'
}).round(2)

segment_profitability.columns = ['_'.join(col).strip() for col in segment_profitability.columns.values]
segment_profitability = segment_profitability.rename(columns={
    'conv_14d_total_spent_in_window_mean': 'avg_revenue_14d',
    'conv_14d_total_spent_in_window_sum': 'total_revenue_14d',
    'ret_30d_retention_total_spent_mean': 'avg_revenue_30d',
    'ret_30d_retention_total_spent_sum': 'total_revenue_30d',
    'conv_14d_conversion_mean': 'conversion_rate',
    'conv_14d_offer_completed_mean': 'offer_completion_rate',
    'ret_30d_retention_active_mean': 'retention_rate',
    'prev_view_rate_mean': 'avg_view_rate',
    'prev_completion_rate_mean': 'avg_completion_rate',
    'customer_id_count': 'offer_instances'
})

print("\n SEGMENT PROFITABILITY METRICS:")
print("=" * 80)
display(segment_profitability)

# Calculate Customer Lifetime Value proxy
segment_profitability['clv_proxy'] = (
    segment_profitability['avg_revenue_30d'] *
    segment_profitability['retention_rate'] * 12  # Annualized
)

print("\n CUSTOMER LIFETIME VALUE (Proxy - Annualized):")
print("=" * 80)
display(segment_profitability[['avg_revenue_30d', 'retention_rate', 'clv_proxy']].sort_values('clv_proxy', ascending=False))

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#F7DC6F']

# 1. Average Revenue by Segment
segment_profitability['avg_revenue_14d'].sort_values().plot(
    kind='barh', ax=axes[0, 0], color=colors, edgecolor='black'
)
axes[0, 0].set_title('Average Revenue per Offer (14d)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Average Revenue ($)')
axes[0, 0].grid(axis='x', alpha=0.3)

# 2. Conversion Rate by Segment
segment_profitability['conversion_rate'].sort_values().plot(
    kind='barh', ax=axes[0, 1], color=colors, edgecolor='black'
)
axes[0, 1].set_title('Conversion Rate by Segment', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Conversion Rate')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Retention Rate by Segment
segment_profitability['retention_rate'].sort_values().plot(
    kind='barh', ax=axes[1, 0], color=colors, edgecolor='black'
)
axes[1, 0].set_title('Retention Rate by Segment', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Retention Rate')
axes[1, 0].grid(axis='x', alpha=0.3)

# 4. CLV Proxy by Segment
segment_profitability['clv_proxy'].sort_values().plot(
    kind='barh', ax=axes[1, 1], color=colors, edgecolor='black'
)
axes[1, 1].set_title('Customer Lifetime Value (Proxy)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('CLV Proxy ($)')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Profitability analysis complete!")

Segment Response to Different Offer Types

In [ ]:
"""
Analyze how different segments respond to different offer types
"""

print("=" * 80)
print(" SEGMENT RESPONSE TO OFFER TYPES")
print("=" * 80)

# Conversion rate by segment and offer type
segment_offer_response = modeling_data.groupby(['segment_name', 'offer_type_simple']).agg({
    'conv_14d_conversion': 'mean',
    'conv_14d_offer_completed': 'mean',
    'conv_14d_total_spent_in_window': 'mean',
    'ret_30d_retention_active': 'mean',
    'customer_id': 'count'
}).round(3)

segment_offer_response.columns = ['conversion_rate', 'completion_rate', 'avg_revenue', 'retention_rate', 'count']

print("\n CONVERSION RATE BY SEGMENT × OFFER TYPE:")
print("=" * 80)
conversion_pivot = segment_offer_response['conversion_rate'].unstack()
display(conversion_pivot)

print("\n AVERAGE REVENUE BY SEGMENT × OFFER TYPE:")
print("=" * 80)
revenue_pivot = segment_offer_response['avg_revenue'].unstack()
display(revenue_pivot)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap: Conversion Rate
import seaborn as sns
sns.heatmap(conversion_pivot, annot=True, fmt='.3f', cmap='YlOrRd',
            ax=axes[0], cbar_kws={'label': 'Conversion Rate'})
axes[0].set_title('Conversion Rate by Segment × Offer Type', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Offer Type')
axes[0].set_ylabel('Segment')

# Heatmap: Average Revenue
sns.heatmap(revenue_pivot, annot=True, fmt='.2f', cmap='Greens',
            ax=axes[1], cbar_kws={'label': 'Avg Revenue ($)'})
axes[1].set_title('Average Revenue by Segment × Offer Type', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Offer Type')
axes[1].set_ylabel('Segment')

plt.tight_layout()
plt.show()

print("\n INSIGHTS:")
print("=" * 80)
print("Use this matrix to determine:")
print("• Which offer types work best for each segment")
print("• Which segments are most valuable for each offer type")
print("• Personalization strategies for the uplift model")

print("\n Segment × Offer analysis complete!")

In [ ]:
"""
Save all segmentation results
"""

print("=" * 80)
print(" SAVING SEGMENTATION RESULTS")
print("=" * 80)

# Save customer-level data with segments
customer_level_data.to_csv('cleaned_data/customer_segments.csv', index=False)
print(" Saved: customer_segments.csv")

# Save segment profiles
segment_profiles.to_csv('cleaned_data/segment_profiles.csv')
print(" Saved: segment_profiles.csv")

# Save segment profitability
segment_profitability.to_csv('cleaned_data/segment_profitability.csv')
print(" Saved: segment_profitability.csv")

# Save modeling data with segments
modeling_data.to_csv('cleaned_data/modeling_data_with_segments.csv', index=False)
print(" Saved: modeling_data_with_segments.csv")

modeling_data.to_pickle('cleaned_data/modeling_data_with_segments.pkl')
print(" Saved: modeling_data_with_segments.pkl")

# Save the scaler and kmeans model
import pickle

with open('cleaned_data/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(" Saved: scaler.pkl")

with open('cleaned_data/kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans_final, f)
print("Saved: kmeans_model.pkl")

print("\n All segmentation results saved!")
print(f"   Location: cleaned_data/")

Define Churn Target Variable

In [ ]:
"""
Define churn target variable
"""

print("=" * 80)
print(" DEFINING CHURN TARGET VARIABLE")
print("=" * 80)

# Churn = 1 if customer is NOT retained (ret_30d_retention_active = 0)
modeling_data['churn_30d'] = (1 - modeling_data['ret_30d_retention_active']).astype(int)

print(f"\n Churn Definition:")
print("   Churn = 1: Customer has NO transactions in next 30 days")
print("   Churn = 0: Customer has at least 1 transaction in next 30 days")

# Check distribution
print(f"\n Churn Distribution:")
churn_dist = modeling_data['churn_30d'].value_counts()
print(churn_dist)

churn_rate = modeling_data['churn_30d'].mean() * 100
print(f"\n Overall Churn Rate: {churn_rate:.2f}%")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Count plot
churn_dist.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'], edgecolor='black')
axes[0].set_title('Churn Distribution (Count)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Churn Status')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Active (0)', 'Churned (1)'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Add count labels
for i, v in enumerate(churn_dist):
    axes[0].text(i, v + 1000, f'{v:,}\n({v/len(modeling_data)*100:.1f}%)',
                 ha='center', fontsize=11, fontweight='bold')

# Churn rate by segment
churn_by_segment = modeling_data.groupby('segment_name')['churn_30d'].mean() * 100
churn_by_segment.plot(kind='bar', ax=axes[1], color='#e74c3c', edgecolor='black')
axes[1].set_title('Churn Rate by Segment', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Segment')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].axhline(churn_rate, color='blue', linestyle='--', linewidth=2, label='Overall Avg')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Churn target variable defined!")

Select Features for Churn Prediction

In [ ]:
"""
Select features for churn prediction
"""

print("=" * 80)
print("SELECTING FEATURES FOR CHURN PREDICTION")
print("=" * 80)

# Select predictive features (everything BEFORE the outcome window)
churn_features = [
    # RFM features (most important)
    'rfm_lifetime_recency_days',
    'rfm_lifetime_frequency',
    'rfm_lifetime_monetary_total',
    'rfm_lifetime_monetary_mean',
    'rfm_lifetime_monetary_max',
    'rfm_lifetime_monetary_std',

    # Recent behavior (30d, 90d)
    'rfm_30d_recency_days',
    'rfm_30d_frequency',
    'rfm_30d_monetary_total',
    'rfm_90d_recency_days',
    'rfm_90d_frequency',
    'rfm_90d_monetary_total',

    # Offer engagement
    'prev_offers_received',
    'prev_offers_viewed',
    'prev_offers_completed',
    'prev_view_rate',
    'prev_completion_rate',
    'prev_completion_given_view_rate',

    # Demographics
    'age_clean',
    'income_clean',
    'tenure_at_offer',
    'gender',

    # Offer characteristics
    'offer_type_simple',
    'reward',
    'min_spend_required',
    'duration_days',

    # Time features
    'hour_of_day',
    'day_of_week',
    'is_weekend',

    # Derived features
    'customer_value_score',
    'engagement_score',
    'is_recently_active',
    'is_high_spender',

    # Segment
    'segment'
]

print(f"\n Selected {len(churn_features)} features for churn prediction:")
for i, feat in enumerate(churn_features, 1):
    print(f"   {i:2d}. {feat}")

# Check for missing values
print(f"\n Checking for missing values...")
missing = modeling_data[churn_features].isnull().sum()
missing = missing[missing > 0]

if len(missing) > 0:
    print(f"\n  Features with missing values:")
    print(missing)
    print(f"\n   Will handle these during preprocessing")
else:
    print(" No missing values in selected features!")

print("\n Features selected!")

In [ ]:
"""
Prepare data for churn prediction modeling
"""

print("=" * 80)
print(" PREPARING DATA FOR MODELING")
print("=" * 80)

# Create a copy for modeling
churn_model_data = modeling_data.copy()

# First, let's see what features we actually have
print("\n Checking available features...")
available_features = []
missing_features = []

for feat in churn_features:
    if feat in churn_model_data.columns:
        available_features.append(feat)
    else:
        missing_features.append(feat)

if len(missing_features) > 0:
    print(f"  Features not in dataframe: {missing_features}")
    print(f"   Removing these from feature list...")

# Update churn_features to only include available features
churn_features = available_features.copy()

print(f" Using {len(churn_features)} available features")

# Handle categorical variables
print("\n Encoding categorical variables...")

# One-hot encode categorical features
categorical_features = ['gender', 'offer_type_simple']
encoded_cols = []

for cat_feat in categorical_features:
    if cat_feat in churn_features:
        print(f"   Encoding {cat_feat}...")
        dummies = pd.get_dummies(churn_model_data[cat_feat], prefix=cat_feat, drop_first=True)
        churn_model_data = pd.concat([churn_model_data, dummies], axis=1)
        encoded_cols.extend(dummies.columns.tolist())

        # Remove original categorical feature
        churn_features.remove(cat_feat)

# Add encoded columns to feature list
churn_features.extend(encoded_cols)

print(f" Categorical variables encoded")
print(f"   Total features now: {len(churn_features)}")

# Verify all features exist
print("\n Final feature verification...")
final_features = []
for feat in churn_features:
    if feat in churn_model_data.columns:
        final_features.append(feat)
    else:
        print(f"     Skipping missing feature: {feat}")

churn_features = final_features
print(f" Final feature count: {len(churn_features)}")

# Handle missing values
print("\n🔧 Handling missing values...")
print(f"\nMissing values by feature:")
missing_counts = churn_model_data[churn_features].isnull().sum()
missing_counts = missing_counts[missing_counts > 0]

if len(missing_counts) > 0:
    print(missing_counts)

    # Fill missing values
    for feat in churn_features:
        if churn_model_data[feat].isnull().any():
            # Check data type
            if churn_model_data[feat].dtype in ['float64', 'int64']:
                # Fill with median for numeric
                fill_value = churn_model_data[feat].median()
                if pd.isna(fill_value):
                    fill_value = 0
                churn_model_data[feat].fillna(fill_value, inplace=True)
            else:
                # Fill with mode for categorical
                mode_val = churn_model_data[feat].mode()
                fill_value = mode_val[0] if len(mode_val) > 0 else 0
                churn_model_data[feat].fillna(fill_value, inplace=True)

            print(f"   Filled {feat}")
else:
    print("No missing values!")

print(f"\n Missing values handled")

# Prepare X and y
X = churn_model_data[churn_features].copy()
y = churn_model_data['churn_30d'].copy()

# Final safety check - replace any remaining NaN with 0
X = X.fillna(0)

print(f"\n Final verification:")
print(f"   X has NaN: {X.isnull().any().any()}")
print(f"   y has NaN: {y.isnull().any()}")

print(f"\n Modeling Dataset:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Features: {X.shape[1]}")
print(f"   Samples: {X.shape[0]:,}")

# Check class balance
print(f"\n Class Distribution:")
print(y.value_counts())
print(f"\n   Churn rate: {y.mean()*100:.2f}%")

print("\n Data prepared for modeling!")

In [ ]:
"""
Split data into train and test sets
"""

print("=" * 80)
print("  TRAIN-TEST SPLIT")
print("=" * 80)

from sklearn.model_selection import train_test_split

# Split the data (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class balance
)

print(f"\n Data Split:")
print(f"   Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n Class Distribution in Train Set:")
print(y_train.value_counts())
print(f"   Churn rate: {y_train.mean()*100:.2f}%")

print(f"\n Class Distribution in Test Set:")
print(y_test.value_counts())
print(f"   Churn rate: {y_test.mean()*100:.2f}%")

# Scale features
print("\n  Scaling features...")
from sklearn.preprocessing import StandardScaler

scaler_churn = StandardScaler()
X_train_scaled = scaler_churn.fit_transform(X_train)
X_test_scaled = scaler_churn.transform(X_test)

print(f" Features scaled using StandardScaler")

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("\n Train-test split complete!")

In [ ]:
"""
Train Logistic Regression model (baseline)
"""

print("=" * 80)
print(" TRAINING LOGISTIC REGRESSION (BASELINE MODEL)")
print("=" * 80)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score
)

print("\n Training Logistic Regression...")

# Train model
lr_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # Handle class imbalance
    solver='lbfgs'
)

lr_model.fit(X_train_scaled, y_train)

print(" Model trained!")

# Predictions
y_train_pred_lr = lr_model.predict(X_train_scaled)
y_test_pred_lr = lr_model.predict(X_test_scaled)

y_train_proba_lr = lr_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate
print("\n LOGISTIC REGRESSION PERFORMANCE:")
print("=" * 80)

print("\n Training Set:")
print(classification_report(y_train, y_train_pred_lr, target_names=['Active', 'Churned']))
print(f"ROC-AUC Score: {roc_auc_score(y_train, y_train_proba_lr):.4f}")

print("\n Test Set:")
print(classification_report(y_test, y_test_pred_lr, target_names=['Active', 'Churned']))
test_auc_lr = roc_auc_score(y_test, y_test_proba_lr)
print(f"ROC-AUC Score: {test_auc_lr:.4f}")

# Confusion Matrix
print("\n Confusion Matrix (Test Set):")
cm_lr = confusion_matrix(y_test, y_test_pred_lr)
print(cm_lr)
print(f"\n   True Negatives (TN): {cm_lr[0,0]:,}")
print(f"   False Positives (FP): {cm_lr[0,1]:,}")
print(f"   False Negatives (FN): {cm_lr[1,0]:,}")
print(f"   True Positives (TP): {cm_lr[1,1]:,}")

print("\n Logistic Regression evaluation complete!")

In [ ]:
"""
Train XGBoost model (high performance)
"""

print("=" * 80)
print(" TRAINING XGBOOST MODEL")
print("=" * 80)

import xgboost as xgb

print("\n Training XGBoost classifier...")

# Train XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss',
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),  # Handle imbalance
    use_label_encoder=False
)

xgb_model.fit(X_train_scaled, y_train)

print(" XGBoost model trained!")

# Predictions
y_train_pred_xgb = xgb_model.predict(X_train_scaled)
y_test_pred_xgb = xgb_model.predict(X_test_scaled)

y_train_proba_xgb = xgb_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate
print("\n XGBOOST PERFORMANCE:")
print("=" * 80)

print("\n Training Set:")
print(classification_report(y_train, y_train_pred_xgb, target_names=['Active', 'Churned']))
print(f"ROC-AUC Score: {roc_auc_score(y_train, y_train_proba_xgb):.4f}")

print("\n Test Set:")
print(classification_report(y_test, y_test_pred_xgb, target_names=['Active', 'Churned']))
test_auc_xgb = roc_auc_score(y_test, y_test_proba_xgb)
print(f"ROC-AUC Score: {test_auc_xgb:.4f}")

# Confusion Matrix
print("\n Confusion Matrix (Test Set):")
cm_xgb = confusion_matrix(y_test, y_test_pred_xgb)
print(cm_xgb)
print(f"\n   True Negatives (TN): {cm_xgb[0,0]:,}")
print(f"   False Positives (FP): {cm_xgb[0,1]:,}")
print(f"   False Negatives (FN): {cm_xgb[1,0]:,}")
print(f"   True Positives (TP): {cm_xgb[1,1]:,}")

print("\n XGBoost evaluation complete!")

In [ ]:
"""
Compare Logistic Regression vs XGBoost
"""

print("=" * 80)
print(" MODEL COMPARISON")
print("=" * 80)

# Create comparison dataframe
model_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'XGBoost'],
    'Test ROC-AUC': [test_auc_lr, test_auc_xgb],
})

print("\n Model Performance Comparison:")
display(model_comparison)

# Visualize ROC curves
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ROC Curve
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_test_proba_lr)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_test_proba_xgb)

axes[0].plot(fpr_lr, tpr_lr, linewidth=2, label=f'Logistic Regression (AUC={test_auc_lr:.3f})', color='#3498db')
axes[0].plot(fpr_xgb, tpr_xgb, linewidth=2, label=f'XGBoost (AUC={test_auc_xgb:.3f})', color='#e74c3c')
axes[0].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve Comparison', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
precision_lr, recall_lr, _ = precision_recall_curve(y_test, y_test_proba_lr)
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, y_test_proba_xgb)

ap_lr = average_precision_score(y_test, y_test_proba_lr)
ap_xgb = average_precision_score(y_test, y_test_proba_xgb)

axes[1].plot(recall_lr, precision_lr, linewidth=2, label=f'Logistic Regression (AP={ap_lr:.3f})', color='#3498db')
axes[1].plot(recall_xgb, precision_xgb, linewidth=2, label=f'XGBoost (AP={ap_xgb:.3f})', color='#e74c3c')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve Comparison', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determine best model
best_model_name = 'XGBoost' if test_auc_xgb > test_auc_lr else 'Logistic Regression'
best_model = xgb_model if test_auc_xgb > test_auc_lr else lr_model
best_proba = y_test_proba_xgb if test_auc_xgb > test_auc_lr else y_test_proba_lr

print(f"\n BEST MODEL: {best_model_name}")
print(f"   Test ROC-AUC: {max(test_auc_lr, test_auc_xgb):.4f}")

print("\n Model comparison complete!")

In [ ]:
"""
Analyze feature importance from XGBoost
"""

print("=" * 80)
print(" FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n Top 20 Most Important Features:")
print("=" * 80)
display(feature_importance.head(20))

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))

top_features = feature_importance.head(20)
ax.barh(range(len(top_features)), top_features['importance'], color='#3498db', edgecolor='black')
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Top 20 Feature Importances (XGBoost)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Feature importance analysis complete!")

In [ ]:
"""
Generate SHAP explanations for model interpretability
"""

print("=" * 80)
print(" GENERATING SHAP EXPLANATIONS")
print("=" * 80)

import shap

print("\n Computing SHAP values (this may take a few minutes)...")

# Create SHAP explainer
explainer = shap.TreeExplainer(xgb_model)

# Calculate SHAP values for test set (use a sample if too large)
sample_size = min(1000, len(X_test_scaled))
X_test_sample = X_test_scaled.sample(n=sample_size, random_state=42)

print(f"   Computing SHAP values for {sample_size} test samples...")
shap_values = explainer.shap_values(X_test_sample)

print(" SHAP values computed!")

# Summary plot
print("\n Generating SHAP summary plot...")
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_sample, plot_type="bar", show=False)
plt.title('SHAP Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Detailed summary plot
print("\n Generating detailed SHAP summary plot...")
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_sample, show=False)
plt.title('SHAP Summary Plot (Feature Impact)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n SHAP explanations generated!")

print("\n INTERPRETATION:")
print("=" * 80)
print("""
The SHAP plots show:
- Which features are most important for predictions
- How feature values impact predictions (red = high value, blue = low value)
- Positive SHAP value = increases churn probability
- Negative SHAP value = decreases churn probability

Top features typically include:
- Recency (days since last purchase)
- Frequency (transaction count)
- Monetary (spending amount)
- Offer engagement rates
""")

In [ ]:
"""
Generate churn risk scores for all customers in the dataset
"""

print("=" * 80)
print(" GENERATING CHURN RISK SCORES")
print("=" * 80)

print("\n Generating predictions for all data...")

# Scale all data
X_all_scaled = scaler_churn.transform(X)

# Predict churn probabilities
churn_probabilities = xgb_model.predict_proba(X_all_scaled)[:, 1]

# Add to modeling data
modeling_data['churn_risk_score'] = churn_probabilities

print(" Churn risk scores generated!")

# Analyze risk distribution
print("\n Churn Risk Distribution:")
print(modeling_data['churn_risk_score'].describe())

# Risk categories
modeling_data['churn_risk_category'] = pd.cut(
    modeling_data['churn_risk_score'],
    bins=[0, 0.3, 0.6, 1.0],
    labels=['Low Risk', 'Medium Risk', 'High Risk']
)

print("\n Risk Category Distribution:")
print(modeling_data['churn_risk_category'].value_counts())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(modeling_data['churn_risk_score'], bins=50, color='#e74c3c', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Churn Risk Score', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Churn Risk Scores', fontsize=14, fontweight='bold')
axes[0].axvline(modeling_data['churn_risk_score'].median(), color='blue',
                linestyle='--', linewidth=2, label=f'Median: {modeling_data["churn_risk_score"].median():.3f}')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Risk categories
risk_counts = modeling_data['churn_risk_category'].value_counts()
axes[1].bar(range(len(risk_counts)), risk_counts.values,
            color=['#2ecc71', '#f39c12', '#e74c3c'], edgecolor='black')
axes[1].set_xticks(range(len(risk_counts)))
axes[1].set_xticklabels(risk_counts.index, rotation=0)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Churn Risk Categories', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add count labels
for i, v in enumerate(risk_counts.values):
    axes[1].text(i, v + 500, f'{v:,}\n({v/len(modeling_data)*100:.1f}%)',
                 ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Churn risk scores added to dataset!")

In [ ]:
"""
Analyze churn risk by customer segment
"""

print("=" * 80)
print(" CHURN RISK BY SEGMENT")
print("=" * 80)

# Average churn risk by segment
segment_risk = modeling_data.groupby('segment_name').agg({
    'churn_risk_score': ['mean', 'median', 'std'],
    'churn_30d': 'mean',
    'customer_id': 'count'
}).round(3)

segment_risk.columns = ['avg_risk', 'median_risk', 'std_risk', 'actual_churn_rate', 'count']

print("\n Churn Risk by Segment:")
display(segment_risk.sort_values('avg_risk', ascending=False))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Average risk by segment
segment_risk['avg_risk'].sort_values().plot(
    kind='barh', ax=axes[0], color='#e74c3c', edgecolor='black'
)
axes[0].set_xlabel('Average Churn Risk Score', fontsize=12)
axes[0].set_title('Average Churn Risk by Segment', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Compare predicted vs actual
comparison = segment_risk[['avg_risk', 'actual_churn_rate']].sort_values('avg_risk')
comparison.plot(kind='barh', ax=axes[1], color=['#e74c3c', '#3498db'], edgecolor='black')
axes[1].set_xlabel('Rate', fontsize=12)
axes[1].set_title('Predicted Risk vs Actual Churn Rate', fontsize=14, fontweight='bold')
axes[1].legend(['Predicted Risk', 'Actual Churn Rate'])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Segment analysis complete!")

In [ ]:
"""
Save churn prediction models and results
"""

print("=" * 80)
print(" SAVING CHURN MODELS AND RESULTS")
print("=" * 80)

import pickle

# Save models
with open('cleaned_data/logistic_regression_churn.pkl', 'wb') as f:
    pickle.dump(lr_model, f)
print(" Saved: logistic_regression_churn.pkl")

with open('cleaned_data/xgboost_churn.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
print(" Saved: xgboost_churn.pkl")

# Save scaler
with open('cleaned_data/scaler_churn.pkl', 'wb') as f:
    pickle.dump(scaler_churn, f)
print(" Saved: scaler_churn.pkl")

# Save feature list
with open('cleaned_data/churn_features.pkl', 'wb') as f:
    pickle.dump(churn_features, f)
print(" Saved: churn_features.pkl")

# Save modeling data with churn scores
modeling_data.to_csv('cleaned_data/modeling_data_with_churn_scores.csv', index=False)
print(" Saved: modeling_data_with_churn_scores.csv")

modeling_data.to_pickle('cleaned_data/modeling_data_with_churn_scores.pkl')
print(" Saved: modeling_data_with_churn_scores.pkl")

# Save model performance metrics
model_metrics = {
    'logistic_regression': {
        'test_auc': test_auc_lr,
        'test_predictions': y_test_pred_lr,
        'test_probabilities': y_test_proba_lr
    },
    'xgboost': {
        'test_auc': test_auc_xgb,
        'test_predictions': y_test_pred_xgb,
        'test_probabilities': y_test_proba_xgb
    },
    'best_model': best_model_name,
    'feature_importance': feature_importance
}

with open('cleaned_data/churn_model_metrics.pkl', 'wb') as f:
    pickle.dump(model_metrics, f)
print(" Saved: churn_model_metrics.pkl")

print("\n All churn models and results saved!")
print(f"   Location: cleaned_data/")

In [ ]:
"""
Create a summary report of churn risk analysis
"""

print("=" * 80)
print(" CHURN RISK SUMMARY REPORT")
print("=" * 80)

print(f"""
CHURN PREDICTION MODEL SUMMARY
================================

 DATASET:
-----------
Total Samples: {len(X):,}
Training Samples: {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)
Test Samples: {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)
Features: {len(churn_features)}
Target: Churn in next 30 days

 CLASS DISTRIBUTION:
----------------------
Overall Churn Rate: {modeling_data['churn_30d'].mean()*100:.2f}%
Active Customers: {(modeling_data['churn_30d']==0).sum():,} ({(1-modeling_data['churn_30d'].mean())*100:.1f}%)
Churned Customers: {(modeling_data['churn_30d']==1).sum():,} ({modeling_data['churn_30d'].mean()*100:.1f}%)

 MODEL PERFORMANCE:
--------------------
Logistic Regression:
  - Test ROC-AUC: {test_auc_lr:.4f}

XGBoost:
  - Test ROC-AUC: {test_auc_xgb:.4f}

Best Model: {best_model_name}
Best Test ROC-AUC: {max(test_auc_lr, test_auc_xgb):.4f}

 TOP 5 PREDICTIVE FEATURES:
------------------------------
""")

for i, row in feature_importance.head(5).iterrows():
    print(f"   {i+1}. {row['feature']}: {row['importance']:.4f}")

print(f"""
CHURN RISK DISTRIBUTION:
---------------------------
Mean Risk Score: {modeling_data['churn_risk_score'].mean():.3f}
Median Risk Score: {modeling_data['churn_risk_score'].median():.3f}
Std Risk Score: {modeling_data['churn_risk_score'].std():.3f}

Risk Categories:
""")

for cat, count in modeling_data['churn_risk_category'].value_counts().items():
    pct = count / len(modeling_data) * 100
    print(f"  • {cat}: {count:,} customers ({pct:.1f}%)")

print(f"""
 SEGMENT RISK ANALYSIS:
-------------------------
Highest Risk Segment: {segment_risk.sort_values('avg_risk', ascending=False).index[0]}
  - Avg Risk: {segment_risk.sort_values('avg_risk', ascending=False)['avg_risk'].iloc[0]:.3f}
  - Actual Churn: {segment_risk.sort_values('avg_risk', ascending=False)['actual_churn_rate'].iloc[0]:.3f}

Lowest Risk Segment: {segment_risk.sort_values('avg_risk', ascending=True).index[0]}
  - Avg Risk: {segment_risk.sort_values('avg_risk', ascending=True)['avg_risk'].iloc[0]:.3f}
  - Actual Churn: {segment_risk.sort_values('avg_risk', ascending=True)['actual_churn_rate'].iloc[0]:.3f}

KEY INSIGHTS:
----------------
- Churn prediction model successfully identifies at-risk customers
- Model can prioritize retention efforts on high-risk customers
- Segments show different churn risk profiles
- Top features align with business intuition (RFM metrics)
- Predictions are calibrated and ready for use in NBA system

READY FOR STEP 5: UPLIFT MODELING
""")

In [ ]:
"""
Prepare data for uplift modeling
"""

print("=" * 80)
print(" PREPARING DATA FOR UPLIFT MODELING")
print("=" * 80)

# We'll use the modeling_data which has all features and outcomes
uplift_data = modeling_data.copy()

print(f"\n Uplift Dataset:")
print(f"   Total samples: {len(uplift_data):,}")
print(f"   Unique customers: {uplift_data['customer_id'].nunique():,}")

# Treatment variable
print(f"\n Treatment Distribution:")
treatment_dist = uplift_data['offer_type_simple'].value_counts()
print(treatment_dist)
print(f"\nPercentages:")
print((treatment_dist / len(uplift_data) * 100).round(2))

# Outcome variables
print(f"\n Outcome Variables:")
print(f"   Conversion rate: {uplift_data['conv_14d_conversion'].mean()*100:.2f}%")
print(f"   Average revenue (14d): ${uplift_data['conv_14d_total_spent_in_window'].mean():.2f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Treatment distribution
treatment_dist.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
axes[0].set_title('Treatment Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Offer Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Conversion by treatment
conv_by_treatment = uplift_data.groupby('offer_type_simple')['conv_14d_conversion'].mean()
conv_by_treatment.plot(kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
axes[1].set_title('Conversion Rate by Offer Type', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Offer Type')
axes[1].set_ylabel('Conversion Rate')
axes[1].tick_params(axis='x', rotation=0)
axes[1].grid(axis='y', alpha=0.3)
axes[1].axhline(uplift_data['conv_14d_conversion'].mean(), color='red', linestyle='--', label='Overall Avg')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n Data prepared for uplift modeling!")

In [ ]:
"""
Select features for uplift modeling (same as churn features but without treatment)
"""

print("=" * 80)
print(" SELECTING FEATURES FOR UPLIFT MODELS")
print("=" * 80)

# Use same features as churn model, but exclude treatment-related features
uplift_features = [feat for feat in churn_features
                   if 'offer_type' not in feat.lower() and
                   'reward' not in feat.lower() and
                   'min_spend' not in feat.lower() and
                   'duration' not in feat.lower()]

print(f"\n Selected {len(uplift_features)} features for uplift modeling:")
print(f"\nFeature categories:")
print(f"   • RFM features")
print(f"   • Engagement features")
print(f"   • Demographics")
print(f"   • Segment membership")
print(f"   • Time-based features")

# Verify features exist
missing_uplift = [f for f in uplift_features if f not in uplift_data.columns]
if missing_uplift:
    print(f"\n  Removing missing features: {missing_uplift}")
    uplift_features = [f for f in uplift_features if f in uplift_data.columns]

print(f"\n Final feature count: {len(uplift_features)}")

In [ ]:
"""
Build T-Learner models for each treatment
We'll compare each treatment (BOGO, Discount, Informational) vs baseline
"""

print("=" * 80)
print("🎓 BUILDING T-LEARNER UPLIFT MODELS")
print("=" * 80)

from sklearn.ensemble import RandomForestClassifier

# We'll build models comparing each treatment to "Informational" as baseline
# (Informational has lowest cost and can serve as control)

print("\n T-Learner Approach:")
print("   Train separate models for each treatment group")
print("   Uplift = P(Y|Treatment) - P(Y|Control)")

# Define treatments to model
treatments = ['BOGO', 'Discount', 'Informational']
control_group = 'Informational'  # Lowest cost offer

print(f"\n Treatments to evaluate: {treatments}")
print(f"   Control/Baseline: {control_group}")

# Store models
treatment_models = {}
control_models = {}

# Prepare data - fill missing values
uplift_data_clean = uplift_data.copy()
for feat in uplift_features:
    if uplift_data_clean[feat].isnull().any():
        uplift_data_clean[feat].fillna(uplift_data_clean[feat].median(), inplace=True)

# Scale features
from sklearn.preprocessing import StandardScaler
scaler_uplift = StandardScaler()
X_uplift = uplift_data_clean[uplift_features]
X_uplift_scaled = scaler_uplift.fit_transform(X_uplift)
X_uplift_scaled = pd.DataFrame(X_uplift_scaled, columns=uplift_features, index=X_uplift.index)

y_conv = uplift_data_clean['conv_14d_conversion']
y_revenue = uplift_data_clean['conv_14d_total_spent_in_window']

print("\n Training models for each treatment...")

for treatment in treatments:
    print(f"\n🔹 Training models for: {treatment}")

    # Get treatment group data
    treatment_mask = uplift_data_clean['offer_type_simple'] == treatment
    X_treatment = X_uplift_scaled[treatment_mask]
    y_treatment = y_conv[treatment_mask]

    print(f"   Treatment group size: {len(X_treatment):,}")
    print(f"   Conversion rate: {y_treatment.mean()*100:.2f}%")

    # Train model for treatment group
    model_treatment = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    model_treatment.fit(X_treatment, y_treatment)
    treatment_models[treatment] = model_treatment

    print(f"   Model trained for {treatment}")

# Train control model
print(f"\n Training control model: {control_group}")
control_mask = uplift_data_clean['offer_type_simple'] == control_group
X_control = X_uplift_scaled[control_mask]
y_control = y_conv[control_mask]

print(f"   Control group size: {len(X_control):,}")
print(f"   Conversion rate: {y_control.mean()*100:.2f}%")

model_control = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
model_control.fit(X_control, y_control)

print(f"   Control model trained")

print("\n All T-Learner models trained!")

In [ ]:
"""
Calculate uplift scores for each customer-treatment combination
"""

print("=" * 80)
print(" CALCULATING UPLIFT SCORES")
print("=" * 80)

print("\n Generating uplift predictions for all customers...")

# Get predictions for all data
control_proba = model_control.predict_proba(X_uplift_scaled)[:, 1]

# Calculate uplift for each treatment
uplift_scores = {}

for treatment in treatments:
    if treatment == control_group:
        # Control vs itself = 0 uplift
        uplift_scores[treatment] = np.zeros(len(X_uplift_scaled))
    else:
        # Treatment probability - Control probability
        treatment_proba = treatment_models[treatment].predict_proba(X_uplift_scaled)[:, 1]
        uplift = treatment_proba - control_proba
        uplift_scores[treatment] = uplift

    # Add to dataframe
    uplift_data_clean[f'uplift_{treatment}'] = uplift_scores[treatment]

    print(f" Calculated uplift for {treatment}")
    print(f"   Mean uplift: {uplift_scores[treatment].mean():.4f}")
    print(f"   Median uplift: {np.median(uplift_scores[treatment]):.4f}")

print("\n All uplift scores calculated!")

# Visualize uplift distribution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, treatment in enumerate(['BOGO', 'Discount']):  # Skip Informational (all zeros)
    if treatment != control_group:
        axes[i].hist(uplift_scores[treatment], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
        axes[i].set_xlabel('Uplift Score', fontsize=12)
        axes[i].set_ylabel('Frequency', fontsize=12)
        axes[i].set_title(f'Uplift Distribution: {treatment} vs {control_group}', fontsize=12, fontweight='bold')
        axes[i].axvline(0, color='red', linestyle='--', linewidth=2, label='No Effect')
        axes[i].axvline(uplift_scores[treatment].mean(), color='green', linestyle='--',
                       linewidth=2, label=f'Mean: {uplift_scores[treatment].mean():.4f}')
        axes[i].legend()
        axes[i].grid(axis='y', alpha=0.3)

# Comparison plot
uplift_comparison = pd.DataFrame({
    'BOGO': uplift_scores['BOGO'],
    'Discount': uplift_scores['Discount']
})
uplift_comparison.boxplot(ax=axes[2])
axes[2].set_ylabel('Uplift Score', fontsize=12)
axes[2].set_title('Uplift Comparison: BOGO vs Discount', fontsize=12, fontweight='bold')
axes[2].axhline(0, color='red', linestyle='--', linewidth=2, label='No Effect')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Uplift visualization complete!")

In [ ]:
"""
Calculate expected profit considering offer costs
"""

print("=" * 80)
print(" CALCULATING EXPECTED PROFIT")
print("=" * 80)

# Define offer costs (from Step 1)
OFFER_COSTS = {
    'BOGO': 5.0,
    'Discount': 3.0,
    'Informational': 0.5,
    'No Offer': 0.0
}

print(f"\n Offer Costs:")
for offer, cost in OFFER_COSTS.items():
    print(f"   {offer:20s}: ${cost:.2f}")

# Average revenue per conversion
avg_revenue_per_conversion = uplift_data_clean[
    uplift_data_clean['conv_14d_conversion'] == 1
]['conv_14d_total_spent_in_window'].mean()

print(f"\n Average revenue per conversion: ${avg_revenue_per_conversion:.2f}")

# Calculate expected profit for each treatment
for treatment in treatments:
    # Expected incremental conversions
    expected_conversions = uplift_scores[treatment]

    # Expected incremental revenue
    expected_revenue = expected_conversions * avg_revenue_per_conversion

    # Expected profit = Revenue - Cost
    offer_cost = OFFER_COSTS[treatment]
    expected_profit = expected_revenue - offer_cost

    # Add to dataframe
    uplift_data_clean[f'expected_revenue_{treatment}'] = expected_revenue
    uplift_data_clean[f'expected_profit_{treatment}'] = expected_profit

    print(f"\n {treatment}:")
    print(f"   Avg expected revenue: ${expected_revenue.mean():.2f}")
    print(f"   Offer cost: ${offer_cost:.2f}")
    print(f"   Avg expected profit: ${expected_profit.mean():.2f}")
    print(f"   Customers with positive profit: {(expected_profit > 0).sum():,} ({(expected_profit > 0).mean()*100:.1f}%)")

print("\n Expected profit calculated!")

In [ ]:
"""
Generate personalized Next Best Action recommendations
"""

print("=" * 80)
print(" GENERATING NEXT BEST ACTION RECOMMENDATIONS")
print("=" * 80)

print("\n Determining best action for each customer...")

# For each customer, find the action with highest expected profit
profit_columns = [f'expected_profit_{treatment}' for treatment in treatments]

# Find best action
uplift_data_clean['best_action'] = uplift_data_clean[profit_columns].idxmax(axis=1)
uplift_data_clean['best_action'] = uplift_data_clean['best_action'].str.replace('expected_profit_', '')

# Get the expected profit of best action
uplift_data_clean['best_action_profit'] = uplift_data_clean[profit_columns].max(axis=1)

# Get the uplift of best action
uplift_data_clean['best_action_uplift'] = uplift_data_clean.apply(
    lambda row: row[f'uplift_{row["best_action"]}'], axis=1
)

# Only recommend action if expected profit > 0, otherwise "No Offer"
uplift_data_clean['recommended_action'] = uplift_data_clean.apply(
    lambda row: row['best_action'] if row['best_action_profit'] > 0 else 'No Offer',
    axis=1
)

print(" Recommendations generated!")

# Summary
print(f"\n RECOMMENDATION SUMMARY:")
print("=" * 80)
recommendation_dist = uplift_data_clean['recommended_action'].value_counts()
print(recommendation_dist)
print(f"\nPercentages:")
print((recommendation_dist / len(uplift_data_clean) * 100).round(2))

# Average expected profit by recommended action
profit_by_action = uplift_data_clean.groupby('recommended_action')['best_action_profit'].mean()
print(f"\n Average Expected Profit by Recommended Action:")
print(profit_by_action.round(2))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Recommendation distribution
recommendation_dist.plot(kind='bar', ax=axes[0],
                        color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#95a5a6'],
                        edgecolor='black')
axes[0].set_title('Next Best Action Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Recommended Action')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Add count labels
for i, v in enumerate(recommendation_dist):
    axes[0].text(i, v + 500, f'{v:,}\n({v/len(uplift_data_clean)*100:.1f}%)',
                 ha='center', fontsize=10, fontweight='bold')

# Expected profit distribution
uplift_data_clean[uplift_data_clean['best_action_profit'] > 0]['best_action_profit'].hist(
    bins=50, ax=axes[1], color='#2ecc71', edgecolor='black', alpha=0.7
)
axes[1].set_xlabel('Expected Profit ($)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Expected Profit Distribution (Positive Only)', fontsize=12, fontweight='bold')
axes[1].axvline(uplift_data_clean['best_action_profit'].mean(), color='red',
                linestyle='--', linewidth=2,
                label=f'Mean: ${uplift_data_clean["best_action_profit"].mean():.2f}')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Recommendations complete!")

In [ ]:
"""
Analyze NBA recommendations by customer segment
"""

print("=" * 80)
print(" NBA RECOMMENDATIONS BY SEGMENT")
print("=" * 80)

# Recommendations by segment
segment_recommendations = pd.crosstab(
    uplift_data_clean['segment_name'],
    uplift_data_clean['recommended_action'],
    normalize='index'
) * 100

print("\n Recommendation Distribution by Segment (%):")
display(segment_recommendations.round(1))

# Average expected profit by segment
segment_profit = uplift_data_clean.groupby('segment_name').agg({
    'best_action_profit': 'mean',
    'best_action_uplift': 'mean',
    'customer_id': 'count'
}).round(2)
segment_profit.columns = ['avg_expected_profit', 'avg_uplift', 'count']

print("\n Average Expected Profit by Segment:")
display(segment_profit.sort_values('avg_expected_profit', ascending=False))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

segment_recommendations.plot(kind='bar', stacked=True, ax=ax,
                            color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#95a5a6'],
                            edgecolor='black')
ax.set_title('Recommended Actions by Segment (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Segment', fontsize=12)
ax.set_ylabel('Percentage (%)', fontsize=12)
ax.legend(title='Recommended Action', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Segment analysis complete!")

In [ ]:
"""
Calculate overall business impact of NBA system
"""

print("=" * 80)
print(" CALCULATING BUSINESS IMPACT")
print("=" * 80)

# Scenario 1: Random offer assignment (baseline)
random_profit_per_customer = uplift_data_clean[profit_columns].mean().mean()

# Scenario 2: Always send best performing offer overall
best_overall_offer = uplift_data_clean[profit_columns].mean().idxmax().replace('expected_profit_', '')
best_overall_profit = uplift_data_clean[f'expected_profit_{best_overall_offer}'].mean()

# Scenario 3: NBA system (personalized recommendations)
nba_profit = uplift_data_clean['best_action_profit'].mean()

# Scenario 4: No offers at all
no_offer_profit = 0

print("\n BUSINESS IMPACT COMPARISON:")
print("=" * 80)

scenarios = pd.DataFrame({
    'Scenario': [
        '1. No Offers',
        '2. Random Assignment',
        f'3. Best Single Offer ({best_overall_offer})',
        '4. NBA System (Personalized)'
    ],
    'Avg Profit per Customer': [
        no_offer_profit,
        random_profit_per_customer,
        best_overall_profit,
        nba_profit
    ]
})

scenarios['Total Profit (All Customers)'] = scenarios['Avg Profit per Customer'] * len(uplift_data_clean)
scenarios['Lift vs Random (%)'] = ((scenarios['Avg Profit per Customer'] / random_profit_per_customer - 1) * 100).round(1)
scenarios['Lift vs Best Single (%)'] = ((scenarios['Avg Profit per Customer'] / best_overall_profit - 1) * 100).round(1)

display(scenarios)

# Calculate incremental value
incremental_vs_random = (nba_profit - random_profit_per_customer) * len(uplift_data_clean)
incremental_vs_best = (nba_profit - best_overall_profit) * len(uplift_data_clean)

print(f"\n INCREMENTAL VALUE:")
print("=" * 80)
print(f"NBA vs Random Assignment:")
print(f"   Incremental profit: ${incremental_vs_random:,.2f}")
print(f"   Lift: {((nba_profit/random_profit_per_customer - 1)*100):.1f}%")

print(f"\nNBA vs Best Single Offer:")
print(f"   Incremental profit: ${incremental_vs_best:,.2f}")
print(f"   Lift: {((nba_profit/best_overall_profit - 1)*100):.1f}%")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Profit comparison
scenarios_plot = scenarios.set_index('Scenario')['Avg Profit per Customer']
scenarios_plot.plot(kind='barh', ax=axes[0],
                   color=['#95a5a6', '#e67e22', '#3498db', '#2ecc71'],
                   edgecolor='black')
axes[0].set_xlabel('Average Profit per Customer ($)', fontsize=12)
axes[0].set_title('Profit Comparison Across Strategies', fontsize=12, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(scenarios_plot):
    axes[0].text(v + 0.01, i, f'${v:.2f}', va='center', fontsize=10, fontweight='bold')

# Lift comparison
lift_data = scenarios.set_index('Scenario')[['Lift vs Random (%)', 'Lift vs Best Single (%)']].iloc[1:]
lift_data.plot(kind='bar', ax=axes[1], color=['#e67e22', '#3498db'], edgecolor='black')
axes[1].set_ylabel('Lift (%)', fontsize=12)
axes[1].set_title('Lift vs Alternative Strategies', fontsize=12, fontweight='bold')
axes[1].legend(['vs Random', 'vs Best Single Offer'])
axes[1].tick_params(axis='x', rotation=45)
axes[1].axhline(0, color='black', linewidth=1)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Business impact calculated!")

In [ ]:
"""
Identify top customers for each action type
"""

print("=" * 80)
print(" TOP CUSTOMERS BY RECOMMENDED ACTION")
print("=" * 80)

# For each action, show top 10 customers by expected profit
for action in ['BOGO', 'Discount', 'Informational']:
    action_customers = uplift_data_clean[
        uplift_data_clean['recommended_action'] == action
    ].nlargest(10, 'best_action_profit')

    print(f"\n TOP 10 CUSTOMERS FOR {action}:")
    print("=" * 80)

    if len(action_customers) > 0:
        display(action_customers[[
            'customer_id',
            'segment_name',
            'churn_risk_score',
            'best_action_uplift',
            'best_action_profit',
            'rfm_lifetime_frequency',
            'rfm_lifetime_monetary_total'
        ]].head(10))

        print(f"\nSummary:")
        print(f"   Total customers recommended {action}: {(uplift_data_clean['recommended_action']==action).sum():,}")
        print(f"   Avg expected profit: ${action_customers['best_action_profit'].mean():.2f}")
        print(f"   Avg churn risk: {action_customers['churn_risk_score'].mean():.3f}")
    else:
        print(f"   No customers recommended for {action}")

print("\n Top customer analysis complete!")

In [ ]:
"""
Create final NBA recommendations output
"""

print("=" * 80)
print(" CREATING NBA RECOMMENDATIONS OUTPUT")
print("=" * 80)

# Aggregate to customer level (average across all their offer instances)
nba_output = uplift_data_clean.groupby('customer_id').agg({
    # Demographics
    'segment': 'first',
    'segment_name': 'first',
    'age_clean': 'first',
    'income_clean': 'first',
    'gender': 'first',

    # Churn risk
    'churn_risk_score': 'first',
    'churn_risk_category': 'first',

    # RFM
    'rfm_lifetime_frequency': 'first',
    'rfm_lifetime_monetary_total': 'first',
    'rfm_lifetime_recency_days': 'first',

    # NBA recommendations (mode/average)
    'recommended_action': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'best_action_profit': 'mean',
    'best_action_uplift': 'mean',

    # Individual treatment profits (average)
    'expected_profit_BOGO': 'mean',
    'expected_profit_Discount': 'mean',
    'expected_profit_Informational': 'mean',

    # Uplift scores
    'uplift_BOGO': 'mean',
    'uplift_Discount': 'mean',
}).reset_index()

print(f"\n NBA Output Dataset:")
print(f"   Customers: {len(nba_output):,}")
print(f"   Columns: {nba_output.shape[1]}")

# Add top 3 reasons (top features from uplift model)
# For simplicity, we'll use feature importance from earlier
print("\n Adding top predictive features...")

# Get top 3 features for each customer based on their values
# This is simplified - in production you'd use SHAP for individual explanations
top_rfm_features = ['rfm_lifetime_recency_days', 'rfm_lifetime_frequency', 'rfm_lifetime_monetary_total']
nba_output['top_reason_1'] = 'Recency: ' + nba_output['rfm_lifetime_recency_days'].round().astype(str) + ' days'
nba_output['top_reason_2'] = 'Frequency: ' + nba_output['rfm_lifetime_frequency'].round().astype(str) + ' txns'
nba_output['top_reason_3'] = 'Total Spend: $' + nba_output['rfm_lifetime_monetary_total'].round().astype(str)

print(" NBA output created!")

# Display sample
print("\n Sample NBA Recommendations:")
display(nba_output[[
    'customer_id',
    'segment_name',
    'churn_risk_category',
    'recommended_action',
    'best_action_profit',
    'best_action_uplift'
]].head(20))

# Summary by segment and risk
print("\n Recommendations by Segment and Risk:")
summary = nba_output.groupby(['segment_name', 'churn_risk_category', 'recommended_action']).size().unstack(fill_value=0)
display(summary)

print("\n NBA recommendations ready!")

In [ ]:
"""
Evaluate uplift model performance using Qini curves and AUUC
"""

print("=" * 80)
print(" UPLIFT MODEL EVALUATION - QINI CURVES")
print("=" * 80)

print("""
Qini Curve measures how well the model ranks customers by uplift.
It shows the cumulative gain achieved by targeting customers in order of predicted uplift.

AUUC (Area Under Uplift Curve) is the summary metric - higher is better.
""")

def calculate_qini_curve(y_true, treatment, uplift_score, n_bins=10):
    """
    Calculate Qini curve for uplift evaluation

    Parameters:
    -----------
    y_true : array
        Actual outcomes (conversions)
    treatment : array
        Treatment assignment (1 for treatment, 0 for control)
    uplift_score : array
        Predicted uplift scores
    n_bins : int
        Number of bins for the curve

    Returns:
    --------
    x_values, y_values for plotting
    """
    # Sort by uplift score (descending)
    sorted_indices = np.argsort(-uplift_score)
    y_sorted = y_true[sorted_indices]
    treatment_sorted = treatment[sorted_indices]

    # Calculate cumulative gains
    n_total = len(y_sorted)
    bin_size = n_total // n_bins

    x_values = []
    y_values = []

    for i in range(n_bins + 1):
        end_idx = min(i * bin_size, n_total)
        if end_idx == 0:
            x_values.append(0)
            y_values.append(0)
            continue

        # Get data up to this point
        y_bin = y_sorted[:end_idx]
        t_bin = treatment_sorted[:end_idx]

        # Calculate gains for treatment and control groups
        n_treatment = t_bin.sum()
        n_control = len(t_bin) - n_treatment

        if n_treatment > 0 and n_control > 0:
            # Conversions in treatment and control
            conv_treatment = y_bin[t_bin == 1].sum()
            conv_control = y_bin[t_bin == 0].sum()

            # Conversion rates
            rate_treatment = conv_treatment / n_treatment
            rate_control = conv_control / n_control

            # Qini value = incremental conversions
            qini = (rate_treatment - rate_control) * end_idx
        else:
            qini = 0

        x_values.append(end_idx / n_total)
        y_values.append(qini)

    return np.array(x_values), np.array(y_values)

# Evaluate uplift for each treatment vs control
print("\n Calculating Qini curves...")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors_map = {'BOGO': '#FF6B6B', 'Discount': '#4ECDC4'}

for idx, treatment in enumerate(['BOGO', 'Discount']):
    print(f"\n Evaluating {treatment} vs {control_group}...")

    # Get data for this treatment and control
    treatment_mask = uplift_data_clean['offer_type_simple'] == treatment
    control_mask = uplift_data_clean['offer_type_simple'] == control_group
    combined_mask = treatment_mask | control_mask

    y_combined = uplift_data_clean.loc[combined_mask, 'conv_14d_conversion'].values
    t_combined = treatment_mask[combined_mask].astype(int).values
    uplift_combined = uplift_data_clean.loc[combined_mask, f'uplift_{treatment}'].values

    # Calculate Qini curve
    x_qini, y_qini = calculate_qini_curve(y_combined, t_combined, uplift_combined, n_bins=20)

    # Calculate AUUC (Area Under Uplift Curve) using trapezoidal rule
    auuc = np.trapz(y_qini, x_qini)

    print(f"   AUUC: {auuc:.4f}")

    # Plot
    axes[0].plot(x_qini, y_qini, linewidth=2, marker='o', markersize=4,
                label=f'{treatment} (AUUC={auuc:.3f})', color=colors_map[treatment])

# Random model baseline
axes[0].plot([0, 1], [0, 0], 'k--', linewidth=2, label='Random Model')
axes[0].set_xlabel('Fraction of Population Targeted', fontsize=12)
axes[0].set_ylabel('Cumulative Incremental Gain', fontsize=12)
axes[0].set_title('Qini Curves - Uplift Model Performance', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Uplift distribution by bins
for treatment in ['BOGO', 'Discount']:
    uplift_scores_sorted = np.sort(uplift_data_clean[f'uplift_{treatment}'].values)[::-1]
    n = len(uplift_scores_sorted)
    bins = np.array_split(uplift_scores_sorted, 10)
    bin_means = [b.mean() for b in bins]

    axes[1].plot(range(1, 11), bin_means, linewidth=2, marker='o', markersize=6,
                label=treatment, color=colors_map[treatment])

axes[1].set_xlabel('Decile (1=Highest Predicted Uplift)', fontsize=12)
axes[1].set_ylabel('Average Uplift Score', fontsize=12)
axes[1].set_title('Average Uplift by Decile', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='red', linestyle='--', linewidth=1, label='No Effect')

plt.tight_layout()
plt.show()

print("\n Qini curve evaluation complete!")

In [ ]:
"""
Calculate profit gained by targeting top K% of customers
"""

print("=" * 80)
print(" PROFIT @ TOP-K ANALYSIS")
print("=" * 80)

print("\n How much profit do we gain by targeting the top K% of customers?")

# Sort customers by expected profit
nba_output_sorted = nba_output.sort_values('best_action_profit', ascending=False)

# Calculate cumulative profit at different thresholds
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0]  # Top 10%, 20%, etc.

profit_at_k = []

for k in thresholds:
    n_customers = int(len(nba_output_sorted) * k)
    top_k = nba_output_sorted.head(n_customers)

    total_profit = top_k['best_action_profit'].sum()
    avg_profit = top_k['best_action_profit'].mean()

    profit_at_k.append({
        'Top K%': f'{k*100:.0f}%',
        'N Customers': n_customers,
        'Total Profit': total_profit,
        'Avg Profit': avg_profit,
        'Profit per Customer (All)': total_profit / len(nba_output)
    })

profit_at_k_df = pd.DataFrame(profit_at_k)

print("\n Profit@Top-K Results:")
display(profit_at_k_df)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Cumulative profit curve
x_pct = [p['Top K%'] for p in profit_at_k]
y_total = [p['Total Profit'] for p in profit_at_k]

axes[0].plot(range(len(x_pct)), y_total, linewidth=3, marker='o', markersize=8, color='#2ecc71')
axes[0].set_xticks(range(len(x_pct)))
axes[0].set_xticklabels(x_pct)
axes[0].set_xlabel('Top K% of Customers (by Predicted Profit)', fontsize=12)
axes[0].set_ylabel('Total Profit ($)', fontsize=12)
axes[0].set_title('Cumulative Profit by Targeting Top K%', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(y_total):
    axes[0].text(i, v + max(y_total)*0.02, f'${v:,.0f}', ha='center', fontsize=9)

# Average profit per customer
y_avg = [p['Avg Profit'] for p in profit_at_k]

axes[1].bar(range(len(x_pct)), y_avg, color='#3498db', edgecolor='black')
axes[1].set_xticks(range(len(x_pct)))
axes[1].set_xticklabels(x_pct)
axes[1].set_xlabel('Top K% of Customers', fontsize=12)
axes[1].set_ylabel('Average Profit per Customer ($)', fontsize=12)
axes[1].set_title('Average Profit in Top K%', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n INTERPRETATION:")
print("=" * 80)
print(f"• By targeting top 10% of customers, we get ${profit_at_k[0]['Total Profit']:,.2f} in profit")
print(f"• By targeting top 50% of customers, we get ${profit_at_k[4]['Total Profit']:,.2f} in profit")
print(f"• Total profit from all customers: ${profit_at_k[-1]['Total Profit']:,.2f}")

print("\n Profit@Top-K analysis complete!")

In [ ]:
"""
Save all NBA models and outputs
"""

print("=" * 80)
print(" SAVING NBA SYSTEM")
print("=" * 80)

import pickle

# Save uplift models
print("\n Saving uplift models...")
for treatment in treatments:
    if treatment != control_group:
        with open(f'cleaned_data/uplift_model_{treatment}.pkl', 'wb') as f:
            pickle.dump(treatment_models[treatment], f)
        print(f" Saved: uplift_model_{treatment}.pkl")

with open('cleaned_data/uplift_model_control.pkl', 'wb') as f:
    pickle.dump(model_control, f)
print(f" Saved: uplift_model_control.pkl")

# Save scaler
with open('cleaned_data/scaler_uplift.pkl', 'wb') as f:
    pickle.dump(scaler_uplift, f)
print(" Saved: scaler_uplift.pkl")

# Save feature list
with open('cleaned_data/uplift_features.pkl', 'wb') as f:
    pickle.dump(uplift_features, f)
print(" Saved: uplift_features.pkl")

# Save offer costs
with open('cleaned_data/offer_costs.pkl', 'wb') as f:
    pickle.dump(OFFER_COSTS, f)
print(" Saved: offer_costs.pkl")

# Save NBA outputs
nba_output.to_csv('cleaned_data/nba_recommendations.csv', index=False)
print(" Saved: nba_recommendations.csv")

nba_output.to_pickle('cleaned_data/nba_recommendations.pkl')
print("Saved: nba_recommendations.pkl")

# Save full uplift data
uplift_data_clean.to_csv('cleaned_data/uplift_data_complete.csv', index=False)
print(" Saved: uplift_data_complete.csv")

uplift_data_clean.to_pickle('cleaned_data/uplift_data_complete.pkl')
print(" Saved: uplift_data_complete.pkl")

print("\n All NBA system components saved!")
print(f"   Location: cleaned_data/")

In [ ]:
"""
Create a simple API function for getting recommendations
"""

print("=" * 80)
print(" CREATING NBA API FUNCTION")
print("=" * 80)

def get_nba_recommendation(customer_id, nba_output_df):
    """
    Get Next Best Action recommendation for a customer

    Parameters:
    -----------
    customer_id : str
        Customer ID
    nba_output_df : DataFrame
        NBA recommendations dataframe

    Returns:
    --------
    dict : Recommendation details
    """

    if customer_id not in nba_output_df['customer_id'].values:
        return {
            'error': 'Customer not found',
            'customer_id': customer_id
        }

    customer = nba_output_df[nba_output_df['customer_id'] == customer_id].iloc[0]

    # Helper function to convert numpy types to Python types
    def to_python_type(val):
        if pd.isna(val):
            return None
        if isinstance(val, (np.integer, np.int64, np.int32)):
            return int(val)
        if isinstance(val, (np.floating, np.float64, np.float32)):
            return float(val)
        return val

    # Build alternative actions dict - only include columns that exist
    alternative_actions = {}

    for treatment in ['BOGO', 'Discount', 'Informational']:
        uplift_col = f'uplift_{treatment}'
        profit_col = f'expected_profit_{treatment}'

        if uplift_col in customer.index and profit_col in customer.index:
            alternative_actions[treatment] = {
                'uplift': round(float(customer[uplift_col]), 4),
                'profit': round(float(customer[profit_col]), 2)
            }
        else:
            # If columns don't exist, set to 0 (baseline/control)
            alternative_actions[treatment] = {
                'uplift': 0.0,
                'profit': 0.0
            }

    recommendation = {
        'customer_id': str(customer_id),
        'segment': str(customer['segment_name']),
        'churn_risk': str(customer['churn_risk_category']),
        'churn_risk_score': round(float(customer['churn_risk_score']), 3),
        'recommended_action': str(customer['recommended_action']),
        'expected_uplift': round(float(customer['best_action_uplift']), 4),
        'expected_profit': round(float(customer['best_action_profit']), 2),
        'alternative_actions': alternative_actions,
        'customer_profile': {
            'lifetime_transactions': int(customer['rfm_lifetime_frequency']),
            'lifetime_spend': round(float(customer['rfm_lifetime_monetary_total']), 2),
            'days_since_last_purchase': int(customer['rfm_lifetime_recency_days']),
            'age': int(customer['age_clean']) if not pd.isna(customer['age_clean']) else None,
            'income': int(customer['income_clean']) if not pd.isna(customer['income_clean']) else None,
        },
        'top_reasons': [
            str(customer['top_reason_1']),
            str(customer['top_reason_2']),
            str(customer['top_reason_3'])
        ]
    }

    return recommendation

# Test the API function
print("\n Testing NBA API function...")
sample_customer_id = nba_output['customer_id'].iloc[0]
sample_recommendation = get_nba_recommendation(sample_customer_id, nba_output)

print(f"\n Sample Recommendation for Customer: {sample_customer_id}")
print("=" * 80)
import json
print(json.dumps(sample_recommendation, indent=2))

print("\n NBA API function created and tested!")

# Test a few more customers
print("\n Testing with 3 different customers:")
for i in range(3):
    cust_id = nba_output['customer_id'].iloc[i]
    rec = get_nba_recommendation(cust_id, nba_output)
    print(f"\n{i+1}. Customer {cust_id}:")
    print(f"   Segment: {rec['segment']}")
    print(f"   Churn Risk: {rec['churn_risk']}")
    print(f"   Recommended Action: {rec['recommended_action']}")
    print(f"   Expected Profit: ${rec['expected_profit']:.2f}")

print("\n NBA API function ready for deployment!")